# **Let's start to automate some translation task for Magic: the Gathering translation**

> *A bit of context*: every weekend, a group of passionate and cool French judges work for about 1H30 (for free), on translating the newest article on the (awesome) ✨ [**Cranial Insertion** Magic: the Gathering blog](https://www.cranial-insertion.com/), from 🇺 English to 🇫 French.
> I want to automate this task, as much as possible!
> *Let's work more now, to work less in the future!*

Here is what I would like to obtain: an entirely automated pipeline that takes care of helping us to produce the best translation of these English-written articles covering our favorite game (Magic!), in order to help us thanks to coding and AI, and which helps us be more efficient.

I'm thinking about these steps:
1. when one of the North-American author submits their weekly article on Cranial Insertion, something should automatically trigger a script running "on the cloud" (a free/cheap "Google Lambda function" ?) or on the Cranial Insertion hosting server itself,
2. this script should do [like I did last year](https://github.com/Naereen/Assistant-traduction-CranialInsertion-en-Python/blob/master/import-the-latest-article.ipynb), by connecting to the Cranial Insertion admin webpage and download the raw text of the newest article. Then it could be sent to the "cloud", into a new tab in our shared [Google doc](https://docs.google.com/), named by the date of the day.
3. Ideally, this Google doc should be cut in three equally-spacen parts, each with a different color.
4. Then, I want to also do [like I did last year in June](https://github.com/Naereen/Assistant-traduction-CranialInsertion-en-Python/blob/master/translate-card-names.ipynb), to translate as much as possible the names of the Magic cards when they are in tags like `[c]Card Name[/c]` or [c=Card Name]Card Name[/c]`.
5. If possible, a free and efficient LLM specialized on translation from English to French, like HuggingFace's [Helsinki-NLP/opus-mt-tc-big-en-fr](https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-fr) model, could be used to suggest a first draft of a translation, paragraph by paragraph, to help us translate the article.
6. Then when this is done and our shared Google doc has been updated to contain what should be read and translated, a message could be posted to our Discord server automatically, to notify the team of translator that the newest article is ready for translation! See <https://www.makerhacks.com/python-messages-discord/>, which requires a [Discord webhook](https://support.discord.com/hc/en-us/articles/228383668-Intro-to-Webhooks).
   > If all this works fine, we could improve it by automatically affecting three out of the six translators, to a third of the article's content, and notifying only them in the message sent to Discord.

*Note:*
- If running this task automatically can't be done easily, I can write a notebook that can run on Google colab or locally on my machine, and run it manually each week.
- From Google colab, [it is (of course) possible to save a document to Google Drive](https://stackoverflow.com/questions/64808087/how-do-i-save-files-from-google-colab-to-google-drive#64810963), so it must be possible to edit a Google doc.

> More links?
> - See <https://huggingface.co/docs/transformers/tasks/translation> for the documentation I followed initially.
> - See <https://github.com/Naereen/Assistant-traduction-CranialInsertion-en-Python> for the initial project I wrote (documentation in French) in June 2024.

## Small signature and versions of some modules

In [4]:
!pip install watermark
%load_ext watermark
%watermark -v -m -a "Lilian Besson (Naereen)" -gu "Naereen" -g -p tensorflow,transformers,feedparser,requests,beautifulsoup4,tqdm

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: Lilian Besson (Naereen)

Github username: Naereen

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 7.34.0

tensorflow    : 2.18.0
transformers  : 4.50.3
feedparser    : not installed
requests      : 2.32.3
beautifulsoup4: not installed
tqdm          : 4.67.1

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.85+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

Git hash: 



----

## Using Google's colab GPU and setting up the requirements to translate English-to-French
Let's check that the Google Colab notebook runs on a GPU and not a CPU, if possible. Otherwise, inference (translation English to French) will be slower.

In [101]:
running_on_GPU = True
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    running_on_GPU = False
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

Let's install the first requirements:

In [6]:
!pip install sacremoses

Let's use this model <https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-fr>:

In [7]:
from transformers import pipeline
pipe_en2fr = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-fr")

Device set to use cpu


We can then very quickly write a function to translate text, and use it on small sentences:

In [8]:
def en2fr(text):
    return pipe_en2fr(text)[0]['translation_text']

In [9]:
%%time
print(en2fr("The Portuguese teacher is very demanding."))
# expected output: Le professeur de portugais est très exigeant.

Le professeur de portugais est très exigeant.
CPU times: user 1.52 s, sys: 15.6 ms, total: 1.54 s
Wall time: 1.57 s


In [10]:
%%time
print(en2fr("Who are you? I am a LLM running on the Google cloud (Google colab)."))
# expected output: Qui êtes-vous? Je suis un LLM fonctionnant sur le cloud Google (Google colab).

Qui êtes-vous? Je suis un LLM fonctionnant sur le cloud Google (Google colab).
CPU times: user 5.1 s, sys: 133 ms, total: 5.24 s
Wall time: 9.86 s


I'll print paragraphs using this code:

In [25]:
from IPython.display import display, Markdown
def print_paragraph(paragraph):
    return display(Markdown(f"> {paragraph}"))

-----

## Let's work on splitting a paragraph on sentences, translating each of them, and joining the result.

I'm gonna use [the best novel on Earth](https://en.wikisource.org/wiki/The_Cats_of_Ulthar), as an example.

In [98]:
with open("The_Cats_of_Ulthar.txt", "r") as file:
    raw_text = file.read()

print_paragraph(raw_text[:2000])

> Weird Tales (vol. 7, no. 2) — The Cats of Ulthar


					H. P. Lovecraft





1926





Exported from Wikisource on April 6, 2025





A Fantastic Tale Is

The Cats of Ulthar



By H. P. Lovecraft



It is said that in Ulthar, which lies beyond the river Skai, no man may kill a cat; and this I can verily believe as I gaze upon him who sitteth purring before the fire. For the cat is cryptic, and close to strange things which men can not see. He is the soul of antique Ægyptus, and bearer of tales from forgotten cities in Meroë and Ophir. He is the kin of the jungle's lords, and heir to the secrets of hoary and sinister Africa. The Sphinx is his cousin, and he speaks her language; but he is more ancient than the Sphinx, and remembers that which she hath forgotten.

In Ulthar, before ever the burgesses forbade the killing of cats, there dwelt an old cotter and his wife who delighted to trap and slay the cats of their neighbors. Why they did this I know not; save that many hate the voice of the cat in the night, and take it ill that cats should run stealthily about yards and gardens at twilight. But whatever the reason, this old man and woman took pleasure in trapping and slaying every cat which came near to their hovel; and from some of the sounds heard after dark, many villagers fancied that the manner of slaying was exceedingly peculiar. But the villagers did not discuss such things with the old man and his wife; because of the habitual expression on the withered faces of the two, and because their cottage was so small and so darkly hidden under spreading oaks at the back of a neglected yard. In truth, much as the owners of cats hated these odd folk, they feared them more; and instead of berating them as brutal assassins, merely took care that no cherished pet or mouser should stray toward the remote hovel under the dark trees. When through some unavoidable oversight a cat was missed, and sounds heard after dark, the loser would lament impotently; or console himself by

Let's improve the `en2fr` translation function:

In [11]:
def en2fr(text, split_length=350):
    """ Translate <text> from English to French.

    EXPERIMENTAL: If its size is larger than <split_length>, split it by sentences, translate each separately, then join the result.
    """
    if len(text) <= split_length:
        return pipe_en2fr(text)[0]['translation_text']
    else:
        sentences = text.split(".")
        translated_sentences = []
        for sentence in sentences:
            if len(sentence) > 0:
                text = sentence
                translated_text = pipe_en2fr(text)[0]['translation_text']
                translated_sentences.append(translated_text)
        return ".".join(translated_sentences)

Translating directly a long text will simply cut it:

In [99]:
print_paragraph(pipe_en2fr(raw_text[:2000])[0]['translation_text'])

Your input_length: 478 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


> Contes bizarres (vol. 7, n ° 2) - Les chats d'Ulthar H. P. L'amour est caché dans les chattes sauvages; L'amour est caché dans les chattes sauvages; L'amour est caché dans les chattes sauvages; L'amour est caché dans les chattes sauvages; L'amour est caché dans les chattes sauvages; L'amour est caché dans les chattes sauvages; L'amour est caché dans les chats sauvages.

In [101]:
print_paragraph(raw_text[:2000])
raw_text_fr = en2fr(raw_text[:2000])
print_paragraph(raw_text_fr)

> Weird Tales (vol. 7, no. 2) — The Cats of Ulthar


					H. P. Lovecraft





1926





Exported from Wikisource on April 6, 2025





A Fantastic Tale Is

The Cats of Ulthar



By H. P. Lovecraft



It is said that in Ulthar, which lies beyond the river Skai, no man may kill a cat; and this I can verily believe as I gaze upon him who sitteth purring before the fire. For the cat is cryptic, and close to strange things which men can not see. He is the soul of antique Ægyptus, and bearer of tales from forgotten cities in Meroë and Ophir. He is the kin of the jungle's lords, and heir to the secrets of hoary and sinister Africa. The Sphinx is his cousin, and he speaks her language; but he is more ancient than the Sphinx, and remembers that which she hath forgotten.

In Ulthar, before ever the burgesses forbade the killing of cats, there dwelt an old cotter and his wife who delighted to trap and slay the cats of their neighbors. Why they did this I know not; save that many hate the voice of the cat in the night, and take it ill that cats should run stealthily about yards and gardens at twilight. But whatever the reason, this old man and woman took pleasure in trapping and slaying every cat which came near to their hovel; and from some of the sounds heard after dark, many villagers fancied that the manner of slaying was exceedingly peculiar. But the villagers did not discuss such things with the old man and his wife; because of the habitual expression on the withered faces of the two, and because their cottage was so small and so darkly hidden under spreading oaks at the back of a neglected yard. In truth, much as the owners of cats hated these odd folk, they feared them more; and instead of berating them as brutal assassins, merely took care that no cherished pet or mouser should stray toward the remote hovel under the dark trees. When through some unavoidable oversight a cat was missed, and sounds heard after dark, the loser would lament impotently; or console himself by

> Contes étranges (vol..7, non.2) Les Chats d'Ulthar H.P.Lovecraft 1926 Exporté de Wikisource le 6 avril 2025 Un conte fantastique est les chats d'Ulthar Par H.P.Lovecraft Il est dit qu'à Ulthar, qui se trouve au-delà de la rivière Skai, aucun homme ne peut tuer un chat; et cela, je peux vraiment le croire en regardant celui qui ronronne devant le feu..Car le chat est cryptique, et proche de choses étranges que les hommes ne peuvent voir..Il est l'âme de l'gypte antique, et porteur de contes des villes oubliées de Meroë et d'Ophir..Il est le parent des seigneurs de la jungle, et l'héritier des secrets de l'Afrique sinistre et sinistre..Le Sphinx est son cousin, et il parle sa langue; mais il est plus ancien que le Sphinx, et se souvient de ce qu'elle a oublié..A Ulthar, avant même que les bourgeois n'interdisent de tuer des chats, habitaient une vieille cotte et sa femme qui se réjouissaient de piéger et de tuer les chats de leurs voisins..Je ne sais pourquoi ils ont fait cela, si ce n'est que beaucoup détestent la voix du chat dans la nuit, et prennent mal que les chats courent furtivement dans les cours et les jardins au crépuscule..Mais quelle qu’en soit la raison, ce vieil homme et cette vieille femme prirent plaisir à piéger et à tuer tous les chats qui s’approchaient de leur taudis ; et de quelques-uns des bruits entendus après la tombée de la nuit, de nombreux villageois crurent que la manière de tuer était extrêmement particulière..Mais les villageois ne discutaient pas de telles choses avec le vieil homme et sa femme ; à cause de l’expression habituelle sur les visages desséchés des deux, et parce que leur maison était si petite et si sombrement cachée sous des chênes qui s’étendaient à l’arrière d’une cour négligée..En vérité, tout comme les propriétaires de chats détestaient ces gens bizarres, ils les craignaient davantage; et au lieu de les réprimander comme des assassins brutaux, ils veillaient simplement à ce qu'aucun animal de compagnie ou souris chéri ne s'égare vers le taudis éloigné sous les arbres sombres..Quand, par quelque oubli inévitable, on manquait un chat, et qu'on entendait des sons après la tombée de la nuit, le perdant se lamentait impuissantment; ou se consolerait par

In [102]:
%%time
print_paragraph(raw_text[:1000])
raw_text_fr = en2fr(raw_text)
print_paragraph(raw_text_fr[:1000])

> Weird Tales (vol. 7, no. 2) — The Cats of Ulthar


					H. P. Lovecraft





1926





Exported from Wikisource on April 6, 2025





A Fantastic Tale Is

The Cats of Ulthar



By H. P. Lovecraft



It is said that in Ulthar, which lies beyond the river Skai, no man may kill a cat; and this I can verily believe as I gaze upon him who sitteth purring before the fire. For the cat is cryptic, and close to strange things which men can not see. He is the soul of antique Ægyptus, and bearer of tales from forgotten cities in Meroë and Ophir. He is the kin of the jungle's lords, and heir to the secrets of hoary and sinister Africa. The Sphinx is his cousin, and he speaks her language; but he is more ancient than the Sphinx, and remembers that which she hath forgotten.

In Ulthar, before ever the burgesses forbade the killing of cats, there dwelt an old cotter and his wife who delighted to trap and slay the cats of their neighbors. Why they did this I know not; save that many hate the voice of 

> Contes étranges (vol..7, non.2) Les Chats d'Ulthar H.P.Lovecraft 1926 Exporté de Wikisource le 6 avril 2025 Un conte fantastique est les chats d'Ulthar Par H.P.Lovecraft Il est dit qu'à Ulthar, qui se trouve au-delà de la rivière Skai, aucun homme ne peut tuer un chat; et cela, je peux vraiment le croire en regardant celui qui ronronne devant le feu..Car le chat est cryptique, et proche de choses étranges que les hommes ne peuvent voir..Il est l'âme de l'gypte antique, et porteur de contes des villes oubliées de Meroë et d'Ophir..Il est le parent des seigneurs de la jungle, et l'héritier des secrets de l'Afrique sinistre et sinistre..Le Sphinx est son cousin, et il parle sa langue; mais il est plus ancien que le Sphinx, et se souvient de ce qu'elle a oublié..A Ulthar, avant même que les bourgeois n'interdisent de tuer des chats, habitaient une vieille cotte et sa femme qui se réjouissaient de piéger et de tuer les chats de leurs voisins..Je ne sais pourquoi ils ont fait cela, si ce n'e

CPU times: user 20.8 s, sys: 21.1 ms, total: 20.8 s
Wall time: 20.8 s


In [105]:
print(f"The entire novel in English is {len(raw_text)} characters long.")
raw_nb_lines = len(raw_text.split("\n"))
print(f"It has approximately {raw_nb_lines} lines.")
raw_nb_words = len(raw_text.split(' '))
print(f"It has approximately {raw_nb_words} words.")


print(f"The entire novel in French is {len(raw_text_fr)} characters long.")
raw_nb_lines_fr = len(raw_text_fr.split("\n"))
print(f"It has approximately {raw_nb_lines_fr} lines.")
raw_nb_words_fr = len(raw_text_fr.split(' '))
print(f"It has approximately {raw_nb_words_fr} words.")

The entire novel in English is 8880 characters long.
It has approximately 91 lines.
It has approximately 1569 words.
The entire novel in French is 9895 characters long.
It has approximately 1 lines.
It has approximately 1634 words.


This small hack I wrote seems to work efficiently, to translate longer texts!

--------

## Trying to translate automatically some paragraphs from a Cranial Insertion article

In [21]:
%%time
first_paragraph = "[a]Glacial Dragonhunt as a spell doesn't have a target, so you don't announce a target at the time you cast it. If you choose to discard a card during its resolution and that card turns out to be a nonland card, this triggers a reflexive triggered ability that has a target, and you announce its target when you put that ability on the stack.[/a]"
print("The first paragraph in English is:")
print_paragraph(first_paragraph)

first_paragraph_fr = en2fr(first_paragraph)
print("The first paragraph in French is:")
print_paragraph(first_paragraph_fr)

The first paragraph in English is:


> [a]Glacial Dragonhunt as a spell doesn't have a target, so you don't announce a target at the time you cast it. If you choose to discard a card during its resolution and that card turns out to be a nonland card, this triggers a reflexive triggered ability that has a target, and you announce its target when you put that ability on the stack.[/a]

The first paragraph in French is:


> [a]La chasse au dragon glaciale en tant que sort n'a pas de cible, donc vous n'annoncez pas de cible au moment où vous la lancez. Si vous choisissez de défausser une carte pendant sa résolution et que cette carte s'avère être une carte non terrestre, cela déclenche une capacité déclenchée par réflexe qui a une cible, et vous annoncez sa cible lorsque vous mettez cette capacité sur la pile.

CPU times: user 13.8 s, sys: 161 ms, total: 13.9 s
Wall time: 16.1 s


This first try of translating a paragraph from [a recent Cranial Insertion article](https://www.cranial-insertion.com/article/4380) is quite a success!
It's very quick: about 1 second to translate a paragraph, when running on a free GPU in Google Colab!

> CPU times: user 765 ms, sys: 1.34 ms, total: 766 ms
> Wall time: 765 ms

It is quite slower on a CPU (as expected), about 20 times slower:

> CPU times: user 13.8 s, sys: 161 ms, total: 13.9 s
> Wall time: 16.1 s

In [27]:
ratio_cpu_to_gpu = round(16.1 / 0.765, 3)
print(f"It is approximately {ratio_cpu_to_gpu} times slower to use the (entirely Free) CPU than to use the (not-free) cheapest GPU.")

It is approximately 21.046 times slower to use the (entirely Free) CPU than to use the (not-free) cheapest GPU.


I'm pretty certain that this model cannot translate paragraph which are longer than about 370 caracters, from what I tried.
See above, I wrote an improved version of `en2fr`, to split paragraphs into sentences, and translate them one-by-one, if the paragraph is longer than the max lengths.

In [82]:
article_content = """[a]Glacial Dragonhunt as a spell doesn't have a target, so you don't announce a target at the time you cast it. If you choose to discard a card during its resolution and that card turns out to be a nonland card, this triggers a reflexive triggered ability that has a target, and you announce its target when you put that ability on the stack.[/a]

[hr]
[q]Does [c]Bloodletter of Aclazotz[/c] double the value of X for [c]Kotis, the Fangkeeper[/c]'s triggered ability?[/q]

[a]I'm afraid not. Bloodletter of Aclazotz causes your opponent to lose twice the amount of life due the damage that Kotis deals to them, but the X in Kotis's ability is the amount of damage it dealt, not the amount of life that was lost as a result of the damage.[/a]

[hr]
[cleft=Breaching Dragonstorm|TDM|101]When the sky looks like that, RUN![/cleft][q]Speaking of [c]Kotis, the Fangkeeper[/c], what's the time frame for casting the exiled cards?[/q]

[a]Since the ability doesn't specify a duration for the "you may cast" effect, it means [i]right now[/i], during the resolution of Kotis's triggered ability. Any cards that you choose not to cast at that time, or that you can't cast because they lack targets or other prerequisites to cast, simply stay in exile indefinitely and uselessly.[/a]

[hr]
[q]In Two-Headed Giant, does the last part of [c]Betor, Kin to All[/c]'s ability just insta-kill my opposing team?[/q]

[a]As long as the "toughness 40 or greater" condition is met, sure. Both players on your opponents' team simultaneously lose half their life rounded up at the same time, and the game sees their shared life total for each player's life total. For example, if the team's life total is 30, each opponent loses 15 life and their team ends up at 0 life. However, note that Betor only counts the total toughness of the creatures that you control individually. It does not consider the creatures your teammate controls.[/a]

[hr]
[q]If I control [c]The Sibsig Ceremony[/c] and I cast some creature, can I sacrifice it to give [c]Unburied Earthcarver[/c] a counter in response to the Ceremony's trigger and still get a Zombie Cleric token? [/q]

[a]Yes, you can do that. The Ceremony's triggered ability will try and fail to destroy the creature, but the creation of the Zombie Cleric token is not contingent on this destruction, so you still get the token.[/a]

[hr]
[q]Can I sacrifice a creature token to [c]Sidisi, Regent of the Mire[/c]'s activated ability?[/q]

[a]Sure, as long as you can target a creature card with mana value 1 in your graveyard. To activate Sidisi's ability, you first choose some number for X; then you choose an appropriate target, which is a creature card with mana value X+1 in your graveyard; finally you pay the cost, which involves sacrificing a creature with mana value X other than Sidisi. X=0 is a legal choice for the ability, and a creature token has a mana value of 0 (usually, unless it's a copy of something that has a nonzero mana value). [/a]

[hr]
[q]After I draw cards with [c]Narset, Jeskai Waymaster[/c]'s ability, can I cast any of the cards I drew?[/q]

[a]If they're instants or have flash, sure. Narset's ability triggers and resolves in your end step, and players get priority in the end step, so you get the opportunity to cast spells from among the cards you drew.[/a]

[hr]
[q]For chapter III of [c]Awaken the Honored Dead[/c], when do I choose the target? Can I target the card I discarded?[/q]

[a]Choosing to discard a card triggers a reflexive ability that requires a target, so you only choose a target after you discard. This means that you could target and bring back the card you just discarded, provided that it's a creature or land card, but this only benefits you if some triggered ability notices the card entering or leaving the graveyard. Otherwise, you achieve the same end result of the card being in your hand by simply choosing not to discard it in the first place.[/a]

[hr]
[q]Can I target [c]Zurgo's Vanguard[/c] with [c]Smile at Death[/c]'s triggered ability?[/q]

[a]That depends on how many creatures you control. The ability that defines Zurgo's Vanguard's power is a characteristic-defining ability that functions at all times and in all zones, including in your graveyard, so Zurgo's Vanguard's power is only small enough to be a legal target for the ability if you control at most two creatures.[/a]

[hr]
[cright=Call the Spirit Dragons]Who you gonna call?[/cright][q]Does [c]Call the Spirit Dragons[/c]'s ability go on the stack even if I don't control a Dragon? In other words, could I respond to the trigger by casting a Dragon with flash and give it a +1/+1 counter with the ability?[/q]

[a]Sure, that works. Call the Spirit Dragons's ability does not have an intervening-if clause that gets checked at the time the ability triggers. The ability goes on the stack regardless of how many Dragons you control, so you can make more Dragons in response to the ability.[/a]

[hr]
[q]If [c]Effortless Master[/c] is the second spell I cast in a turn, does it get the +1/+1 counters?[/q]

[a]Absolutely. As Effortless Master resolves, the replacement effect checks how many spells you've cast this turn, and you have cast two or more spells this turn, so the condition for Effortless Master to get counters is satisfied.[/a]

[hr]
[q]If [c]New Way Forward[/c] deals an amount of damage that's lethal to my opponent but also causes me to draw more cards than I have left in my library, what happens?[/q]

[a]The game ends in a draw. The delayed triggered ability from New Way Forward's prevention effect deals damage to your opponent and causes you to draw a bunch of cards, and then state-based actions are checked. State-based actions see that your opponent is at 0 or less life and that you've attempted to draw from an empty library, so both of you lose the game at the same time and the game is a draw.[/a]

[hr]
[q]If I control [c]Hollowmurk Siege[/c] with Sultai chosen, do I draw a card when [c]Host of the Hereafter[/c] enters with counters on it?[/q]

[a]Yup! An ability that triggers when a counter is put on a permanent also triggers when a permanent enters with a counter on it. Also note that the ability would ordinarily trigger twice because Host of the Hereafter enters with two counters, but the ability is limited to triggering once each turn.[/a]

[hr]
[q]If [c]Mardu Siegebreaker[/c]'s ability triggers twice due to [c]Panharmonicon[/c] for example, what happens when its attack trigger resolves?[/q]

[a]Thanks to Panharmonicon, there are two cards that fit the description of "the exiled card", and the ability simply performs its actions on both of those cards, so you'll make tapped and attacking token copies of both cards for each opponent.[/a]

[hr]
[q]What happens if [c]Mardu Siegebreaker[/c]'s ability exiles a [c=Cubwarden]mutate pile[/c]?[/q]

[a]You'll get a very similar situation as in the previous question. Mardu Siegebreaker exiles one creature, but it turns into several individual cards in exile. All of those cards are "the exiled card", so you'll make token copies of each of those cards for each opponent.[/a]

[hr]

And that's all the time we have for today's episode. Thanks for reading, and until next time, watch out for stormy weather!

- Carsten Haese
"""

Let's display these paragraphs nicely:

In [108]:
article_paragraphs = article_content.split("\n\n")
for i, paragraph in enumerate(article_paragraphs):
    print(f"{i}-th paragraph (of length {len(paragraph)} letters and {len(paragraph.split(' '))} words)")
    print_paragraph(paragraph)

0-th paragraph (of length 346 letters and 64 words)


> [a]Glacial Dragonhunt as a spell doesn't have a target, so you don't announce a target at the time you cast it. If you choose to discard a card during its resolution and that card turns out to be a nonland card, this triggers a reflexive triggered ability that has a target, and you announce its target when you put that ability on the stack.[/a]

1-th paragraph (of length 123 letters and 15 words)


> [hr]
[q]Does [c]Bloodletter of Aclazotz[/c] double the value of X for [c]Kotis, the Fangkeeper[/c]'s triggered ability?[/q]

2-th paragraph (of length 268 letters and 51 words)


...

> [hr]

30-th paragraph (of length 123 letters and 22 words)


> And that's all the time we have for today's episode. Thanks for reading, and until next time, watch out for stormy weather!

31-th paragraph (of length 16 letters and 3 words)


> - Carsten Haese


Let's try to translate each paragraph:

In [109]:
%%time
article_paragraphs = article_content.split("\n\n")
translated_paragraphs = []
for i, paragraph in enumerate(article_paragraphs):
    print(f"\n{i}-th English paragraph (of length {len(paragraph)} letters and {len(paragraph.split(' '))} words)")
    print_paragraph(paragraph)
    translated_paragraph = en2fr(paragraph)
    print(f"Translated {i}-th paragraph in French (of length {len(translated_paragraph)} letters and {len(translated_paragraph.split(' '))} words)")
    print_paragraph(translated_paragraph)
    translated_paragraphs.append(translated_paragraph)



0-th English paragraph (of length 346 letters and 64 words)


> [a]Glacial Dragonhunt as a spell doesn't have a target, so you don't announce a target at the time you cast it. If you choose to discard a card during its resolution and that card turns out to be a nonland card, this triggers a reflexive triggered ability that has a target, and you announce its target when you put that ability on the stack.[/a]

Translated 0-th paragraph in French (of length 392 letters and 69 words)


> [a]La chasse au dragon glaciale en tant que sort n'a pas de cible, donc vous n'annoncez pas de cible au moment où vous la lancez. Si vous choisissez de défausser une carte pendant sa résolution et que cette carte s'avère être une carte non terrestre, cela déclenche une capacité déclenchée par réflexe qui a une cible, et vous annoncez sa cible lorsque vous mettez cette capacité sur la pile.



1-th English paragraph (of length 123 letters and 15 words)


...



31-th English paragraph (of length 16 letters and 3 words)


> - Carsten Haese


Translated 31-th paragraph in French (of length 15 letters and 3 words)


> - Carsten Haese

CPU times: user 20.1 s, sys: 37.1 ms, total: 20.2 s
Wall time: 20.3 s


It took about 23 seconds to generate a first draft of a translation for a content lengthy like two thirds (2/3) of a usual-size article.
So it should take about 35 seconds to translate a full-size article, that's great!

------

## Failures of the [Helsinki-NLP/opus-mt-tc-big-en-fr](https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-fr) model for translating text about Magic: the Gathering (on Cranial Insertion)

Here are a few points I observed on which this great NLP LLM fails:

- land/nonland => ~~terrestre~~ terrain/non terrain
- you => ~~vous~~ tu (on tutoie le lecteur dans les articles Cranial)
- reflexive trigger => capacité déclenchée ~~par réflexe~~ réflexive
- and most likely MANY MORE!

### TODO: check more examples to list here the failures I observed

### Future goal: fine-tune this model 🪄
One future goal will be to fine-tune the [Helsinki-NLP/opus-mt-tc-big-en-fr](https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-fr) model, using the data of all the previously manually translated Cranial Insertion articles (and maybe other sources), in order to fix these issues and improve the overall quality of the translation.

------

## Getting the text of the latest article

By using the RSS feed of the Cranial Insertion blog, I'll download the raw content of the newest article.

In [12]:
!pip install feedparser
import feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.3 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=24592839b042de3cde9b88d93e062e0a9359f9b39eb3c8921b5672fe251b7c12
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [13]:
rss_url = "https://www.cranial-insertion.com/rss"
print(f"Reading the feed at {rss_url}...")

blog_feed = feedparser.parse(rss_url)
print(f"Succesfully read the feed at {rss_url}, named « {blog_feed.feed.title} »")

Reading the feed at https://www.cranial-insertion.com/rss...
Succesfully read the feed at https://www.cranial-insertion.com/rss, named « Cranial Insertion »


In [14]:
num_article = 0

latest_article_link = blog_feed.entries[num_article].link
print("- The latest article is here:", latest_article_link)

latest_article_title = blog_feed.entries[num_article].title
print("- Its title is:", latest_article_title)

latest_article_published_data = blog_feed.entries[num_article].published
print("- It was published on:", latest_article_published_data)

article_id = latest_article_link.split("/")[-1]
print("- The article ID is:", article_id)

- The latest article is here: http://www.cranial-insertion.com/article/4376
- Its title is: Cranial Insertion: All in Jeopardy (Can you answer in the form of a question?)
- It was published on: Mon, 31 Mar 2025 00:00:00 EDT
- The article ID is: 4376


In [15]:
def get_latest_article_id(num_article=0):
    blog_feed = feedparser.parse(rss_url)
    latest_article_link = blog_feed.entries[num_article].link
    article_id = latest_article_link.split("/")[-1]
    return article_id

print(f"The latest article ID is: {get_latest_article_id()}")

The latest article ID is: 4376


In [16]:
try:
    with open("cranial-insertion.com.cookies", 'r') as file:
        cranial_insertion_cookie = file.read()
    print("Cranial Insertion cookie successfully loaded from local file 'cranial-insertion.com.cookies', it can now be used (it's a secret)")
except FileNotFoundError:
    from google.colab import userdata
    cranial_insertion_cookie = userdata.get('CI_COOKIE')
    print("Cranial Insertion cookie successfully loaded from Google Colab secrets, it can now be used (it's a secret)")

Cranial Insertion cookie successfully loaded from Google Colab secrets, it can now be used (it's a secret)


We need two modules, to get the text of Cranial Insertion articles:

In [17]:
!pip install requests beautifulsoup4
import requests
from bs4 import BeautifulSoup

In [18]:
def get_article_text(article_id=article_id):
    latest_article_edit_url = f"https://www.cranial-insertion.com/staff/articles/{article_id}/edit"
    print(f"Reading the article at URL {latest_article_edit_url} ...")

    cookies = {
        'loggedin': cranial_insertion_cookie,
        'siteLang': 'fr',
    }
    print(f"Using French language and my admin cookie...")

    response = requests.get(latest_article_edit_url, cookies=cookies)
    soup = BeautifulSoup(response.text, 'html.parser')
    print(f"The web page has been read and it's title is « {soup.title} » !")

    latest_article_text = soup.find(id="thisArticleText").get_text()
    return latest_article_text

We can know get the text of any English article, if we know its ID:

I could improve that but there is no need.

In [24]:
list_of_article_id = [
    4380,
    4376,
    4374,
    4371,
    # etc
    # see https://www.cranial-insertion.com/staff/articles list
]

latest_article_text = get_article_text(get_latest_article_id())
print("This article raw text is:")
print_paragraph(latest_article_text)

Reading the article at URL https://www.cranial-insertion.com/staff/articles/4376/edit ...
Using French language and my admin cookie...
The web page has been read and it's title is « <title>Cranial Insertion | Manage Articles</title> » !
This article raw text is:


> [cright=Living Death]"Alex, the answer is "What would be an
example of an oximoron?"[/cright]

Today we have a collection of questions found on the IRC Chatroom #magicjudges-rules and the Facebook group Ask the Judge - [b]Magic[/b]: The Gathering Rules and Policy Questions. Feel free to join us in either or both groups.

If you have any [b]Magic[/b] questions burning at the back of your brain, you can send them to us. We may even use them in a future article. If you have a short question, you can send it to us via our Twitter account at [url=https://twitter.com/CranialTweet]@CranialTweet[/url], and you can send us longer questions at [email]moko@cranialinsertion.com[/email].

[hr]
[Q] I have two [c]Squire[/c]s on the battlefield and I have [c]Custodi Soulbinders[/c] and four copies of [c]Bear Cub[/c] in my graveyard. If I cast [c]Living Death[/c], how many +1/+1 counters will Soulbinders enter with?[/Q]

[A] Custodi Soulbinders will enter with zero +1/+1 counters. Souldbinders and the Bears will enter simultaneously. Soulbinders' first ability is a replacement effect that checks to see how many creatures are on the battlefield just before it enters. At that point there are no creatures on the board, so it gets no counters as it enters.[/A]

[hr]
[Q] My commander is [c]Grimgrin, Corpse-Born[/c]. Grimgrin was [c]Murder[/c]ed last turn and I choose to move him to the command zone. I control [c]Rooftop Storm[/c]. If I try to cast Grimgrin from the command zone, will he cost [m]2[/m] or will he be free?[/Q]

[A] He will cost [m]2[/m]. 
Commander "tax" is an additional cost that must be paid in addition to the normal cost for casting your commander. Rooftop Storm gives you the option of replacing the normal cost of casting your Zombies with [m]0[/m]. This has no effect on additional costs.  [/A]

[hr]
[Q] My opponent exiled [c]Wheel of Fate[/c] with [c]Reckless Impulse[/c]. She now claims she can cast Wheel of Fate because it's in exile. Is that true?[/Q]

[A] Your opponent is wrong. The part of suspend that lets you cast a card with no mana cost is casting it without paying its mana cost, not that it's casting it from exile.[/A]

[hr]
[Q] I control four [c]Bear Cub[/c]s, [c]Agrus Kos, Eternal Soldier[/c], and [c]Thundering Raiju[/c]. None of my creatures are currently modified. I attack with the Raiju and have its attack trigger target Agrus. When Agrus's ability triggers because he became targeted, I pay the cost. After all these triggers resolve, how much damage will Raiju deal to each opponent?[/Q]

[A] The Answer is between 15-19 damage, depending on when you let the trigger targeting Raiju resolve.

When Agrus's trigger resolves, it creates five copies of Raiju's trigger, one for each Bear and one for Raiju, making a total of six Raiju triggers. The last trigger to resolve has to be the one targeting Agrus, and the other five resolve before it in the order of your choosing. 

For our first example, we will have the trigger targeting Raiju resolve first, then the four triggers targeting Bears, and finally the one targeting Agrus.

Raiju gets a +1/+1 counter, no damage is dealt as Raiju only checks modified creatures other than himself. Total damage 0.
Bear #1 gets a +1/+1 counter, 1 damage is dealt as there is only one modified creature other than Raiju. Total damage 1.
Bear #2 gets a +1/+1 counter, 2 damage is dealt as there are now two modified creatures. Total damage 3.
Bear #3 gets a +1/+1 counter, 3 damage is dealt. Total damage 6.
Bear #4 gets a counter, 4 damage is dealt. Total damage of 10.
Agrus get a a counter, 5 damage is dealt. Total damage of 15.

Now let's have the four Bears' triggers resolve first, then Raiju, followed by Agrus.
Bear #1, 1 damage is dealt. Total damage 1.
Bear #2, 2 damage is dealt. Total damage 3.
Bear #3, 3 damage id dealt. Total damage 6.
Bear #4, 4 damage is dealt. Total damage is 10.
Raiju, 4 damage is dealt. Total damage is 14.
Agrus, 5 damage is dealt. Total damage is 19.

The later the trigger targeting Raiju resolves, the more the total damage.
 [/A]

[hr]
[cleft=Grafdigger's Cage]Alex, the answer is "What is used to 
keep the dead from escaping their graves?"[/cleft][Q] Does [c]Grafdigger's Cage[/c] stop [c]Abhorrent Oculus[/c] from manifest-dreading?[/Q]

[A] Well, in the important ways, yes. They still "manifest dread", but some parts they can't complete. Manifest Dread is "Look at the top two cards of your library. Manifest one of them, then put the cards you looked at that were not manifested this way into your graveyard." Since you can't put a creature from your library into play, both cards looked at are put into the graveyard.[/A]

[hr]
[Q] Last turn my opponent stole my [c]Bear Cub[/c] by casting [c]Sower of Temptation[/c]. This turn I used [c]Ovinize[/c] to make the Sower lose all its abilities. I did this because I think I'll get my Bear Cub back, since the Sower no longer has the "steal your creature" ability. My opponent disagrees. Who is right?[/Q]

[A] Your opponent is correct. Removing the enters ability (that creates an effect with a duration) after the ability has resolved, doesn't stop it from functioning.

If it helps, an example that seems more intuitive would be the following: My opponent used [c]Rootwater Matriarch[/c] to steel my [c]Hill Giant[/c] enchanted with [c]Spirit Link[/c]. Killing the Matriarch won't get the Hill Giant back, even though that removes where the steal ability came from. The duration is based on whether or not the Hill Giant is still enchanted.

So with the original question, removing Sower's abilities doesn't stop the steal effect. You have to remove the Sower to gain the creature back because that ends the duration of the effect.[/A]

[hr]
[Q] So continuing with the last question, what if instead of [c]Ovinize[/c]ing the [c]Sower of Temptation[/c] I phased it out with [c]Reality Ripple[/c]?[/Q]

[A] You would get your creature back. If a creature phases out, it's treated as though it's no longer on the battlefield. The duration of its effect has been met, so the effect ends and the stolen creature returns to its last controller, usually its owner.[/A]

[hr]
[Q] I block my opponent's attacking [c]Bear Cub[/c] with [c]Manor Gargoyle[/c]. After combat ends, the opponent then casts [c]Disfigure[/c] targeting my Gargoyle. They claim they just killed my Gargoyle. Is this true?[/Q]

[A] It is not true, your Gargoyle lives through this. Many new players make this same wrong assumption. Damage doesn't reduce toughness. You have a 2/2 Gargoyle with 2 damage marked on it. Because it's indestructible, the state based action that tries to destroy it for having equal or more damage than it has toughness fails to do anything. If they want to get your Gargoyle into the graveyard, they need to give it enough [c=Dismember]-X/-X to make its toughness 0 or less[/c], or force you to [c=Diabolic Edict]sacrifice it[/c].[/A]

[hr]
[Q] If [c]Ragavan, Nimble Pilferer[/c] is my commander, can I cast him for his dash cost from the command zone?[/Q]

[A] Yes, you can. Dash can be used from any zone, so long as you are allowed to cast that card from that same zone normally. [/A]

[hr]
[Q] Following up with the last question, do I have to pay commander tax if I dash out my commander?[/Q]

[A] Yes, you do. Suppose you cast Ragavan normally on turn one, then later he died and returned to the command zone. When you go to dash him he now costs [m]3[/m][m]R[/m] to dash. Then if he dies again and returns to the command zone, the next time you go to cast him he costs either [m]4[/m][m]R[/m] to cast normally or [m]5[/m][m]R[/m] to dash.[/A]

[hr]
[Q] I have eight cards in my hand. I go to my cleanup step and discard [c]Blazing Rootwalla[/c] to get down to maximum hand size. Can I cast the Rootwalla even though I'm in my cleanup step?[/Q]

[A] Yes, you can. During most cleanup steps, no players get priority. However, if an ability triggers during the cleanup step, or a state based action applies during a cleanup step, then the active player gains priority and may cast spells and activate abilities. 

This is when you can cast the Rootwalla, as discarding it to madness puts a trigger on the stack.

A normal priority pass will begin from here and until all players pass in succession with an empty stack, players may add new items to the stack when they have priority. Once all players pass with an empty stack, this cleanup step ends and a new one begins. If that new cleanup ends normally, then the game will move on to the next player's turn.[/A]

[hr]
[Q] While resolving [c]Burning Inquiry[/c] I discarded [c]Basking Rootwalla[/c] and two [c]Mountain[/c]s. My opponent controls a [c]Rest in Peace[/c]. My opponent claims I can't cast the Rootwalla for its Madness cost as it got exiled by Rest in Peace when I discarded it. Is this true?[/Q]

[A] Your opponent is wrong. You can cast your Rootwalla for its madness cost. The Rootwalla hasn't been exiled by Rest in Peace yet. This is because madness doesn't quite work the way most people think it does. If a card with madness gets discarded, it goes to exile instead of being put into its owner's graveyard. Then the next time a player would get priority, a trigger allowing its owner to cast it for its madness cost is put on the stack. When this trigger resolves, the owner may cast the card, but if they don't then the card is put into their graveyard. 

To get back to the original question, Rest in Peace can only exile a discarded Madness card after the owner has decided not to cast it for madness and choses to let it go to the graveyard.
[/A]

[hr]
[Q] My opponent pays [m]3[/m][m]U[/m] to gain level 2 on their [c]Stormchaser's Talent[/c]. Before it becomes level 2, I [c]Boomerang[/c] it to their hand. The opponent says they still get to return an instant or sorcery to their hand, I say they don't, who is right?[/Q]

[A] You are correct, they do not. Gaining a level is an ability that uses the stack and can be responded to. Stormchaser's level 2 ability is a trigger that triggers upon it becoming level 2 and happens only after it becomes level 2. If it's not on the battlefield when its level 2 ability resolves, there is no ability to trigger (nor a permanent to become level 2). [/A]

[hr]
[cright=Bear Cub|PO2]Alex, the answer is "What is another
name for Ursine Child?"[/cright][Q] I enchanted my opponent's [c]Bear Cub[/c] with my [c]Sigil of Sleep[/c]. If the Bear Cub goes unblocked attacking me, who chooses which creature I control gets returned to its owner's hand?[/Q]

[A] You get to choose as you control the Sigil of Sleep.[/A]

[hr]
[Q] I attack with [c]Geist of Saint Traft[/c] while I control [c]Anointed Procession[/c]. When combat ends, will both Angel tokens get exiled or do I get to keep one?[/Q]

[A] Both still get exiled. Saint Traft's triggered ability still made both tokens and tracks them to be exiled at the end of combat. Procession doesn't make tokens, it just changes how many tokens other cards and abilities make.[/A]

[hr]
[Q] I attack with [c]Geist of Saint Traft[/c], but before blocks, I cast [c]Eyes in the Skies[/c] choosing to populate the Angel token. When combat ends, will both Angel tokens get exiled or do I get to keep one?[/Q]

[A] You get to keep the one made by Eyes in the Skies. Unlike Anointed Procession from the last question, Eyes in the Skies makes the token and the token has no link to the ability that made the original Angel token. This includes that the second Angel token is not attacking, is not tapped, and it won't get exiled at the end of combat.[/A]

[hr]
[Q] I don't understand what [c]Emrakul, the World Anew[/c] has protection from. Can you clarify this?[/Q]

[A] Emrakul has protection from spells and separately protection from permanents cast "this turn". This turn is whichever turn it currently is, even if it's been many turns since Emrakul herself was cast. 

So for example, if I cast [c]Visara the Dreadful[/c] and gave her some [c]Lightning Greaves[/c] the same turn I cast her, she can't target and destroy Emrakul that turn. However, if I pass to my opponent's turn, Visara can now kill Emrakul as it's no longer the same turn Visara was cast. If I used [c]Goryo's Vengeance[/c] to reanimate Visara, then Visara could kill Emrakul that same turn as Visara wasn't cast at all.

Emrakul's protection ability acts similar to having protection from Auras. Auras when cast target what they will enchant, so Emrakul is safe from most Auras. However, on later turns, an Aura could be moved from its original target, to enchant Emrakul. For example, you could move [c]Pacifism[/c] from a 1/1 Goblin token to Emrakul using [c]Crown of the Ages[/c]. 
[/A]

[hr]
[Q] I cast [c]Season of Weaving[/c] choosing the two paws mode (to copy my [c]Arcanis the Omnipotent[/c]) and the three paws mode. Will I have to put one Arcanis into my graveyard due to the legend rule before bouncing all nonland and nontoken permanents?[/Q]

[A] You will not and you can not. The legend rule is a state based action. It's never checked in the middle of resolving a spell. You will put the original Arcanis into your hand before Season of Weaving fully resolves. The legend rule doesn't come into play during this situation.[/A]

[hr]
[Q] If I cast [c]Righteous Confluence[/c] choosing mode two once to exile my opponent's [c]Leyline of Sanctity[/c] and mode three twice to gain 10 total life, and the opponent responds by [c]Boomerang[/c]ing the Leyline back to their hand, do I still gain 10 life?[/Q]

[A] You do not. Righteous Confluence had only one target, Leyline of Sanctity, and that target has disappeared. If a spell loses all of its targets by the time it goes to resolve, it fizzles and none of its effects happen. [/A]

[hr]
[Q] I just cast [c]Cultivate[/c] putting a [c]Swamp[/c] onto the battlefield and a [c]Forest[/c] into my hand. I haven't played a land from my hand yet this turn. Can I still play the Forest as my land drop for the turn, or did the Swamp use up my land drop for the turn?[/Q]

[A] You may still play the Forest. Putting a land onto the battlefield with a spell or ability doesn't use up your one land drop per turn.[/A]

[hr]

Each Question in my last article had a hidden reference to a different pop-culture pig. With ten different questions there were ten different pigs. The spoiler below has the answer key to those references.
[spoiler]
Question 1) Charlotte's Web - Wilbur the Pig and Charlotte the Spider
Question 2) Jim Henson's Muppets - Miss Piggy, Kermit the Frog, and Fozzie Bear.
Question 3) The Three Little Pigs and the Big Bad Wolf
Question 4) Angry Birds Video Game (Birds vs Pigs)
Question 5) Looney Tunes - Bugs Bunny, Porky Pig, and Daffy Duck
Question 6) Babe the 1995 Movie - Babe the Pig, Sheep for Babe to herd, and Farmer Arthur Hoggett
Question 7) Legend of Zelda - Link an Elf Warrior, Zelda an Elf Noble, and Ganon (pig form of Ganondorf)
Question 8) The Lion King - Timon the Meerkat, Pumbaa the Warthog, and some Hyenas for them to fight.
Question 9) Toy Story - Woody the Cowboy, Buzz the Spaceman, Rex the T-Rex, and Hamm the Piggy Bank.
Question 10) Winnie the Pooh - Pooh the Bear, Tigger the Tiger, and Piglet the Pig.[/spoiler]

- Justin Hovdenes AKA Hovey
Level 2 Magic Judge
Rapid City, SD

------------------

## Now, let's translate the entire article!

I want to be able to obtain:

- the entire raw text,
- its splitting into paragraphs,
- their translations,

just from a given (or the latest) article ID.

In [28]:
from tqdm.notebook import tqdm

In [88]:
def get_and_translate_article_text(article_id=None, translate=False):

    if article_id is None:
        try:
            article_id = get_latest_article_id()
        except:
            article_id = 4376  # etc, see https://www.cranial-insertion.com/staff/articles for the entire list

    latest_article_text = get_article_text(article_id)

    article_content = latest_article_text
    article_paragraphs = article_content.split("\n\n")
    translated_paragraphs = []

    for i, paragraph in tqdm(enumerate(article_paragraphs)):
        print(f"\n\n{i}-th English paragraph (of length {len(paragraph)} letters and {len(paragraph.split(' '))} words)")
        print_paragraph(paragraph)

        if translate:
            translated_paragraph = en2fr(paragraph)
            print(f"Translated {i}-th paragraph in French (of length {len(translated_paragraph)} letters and {len(translated_paragraph.split(' '))} words)")
            print_paragraph(translated_paragraph)
            translated_paragraphs.append(translated_paragraph)

    return article_id, latest_article_text, article_paragraphs, translated_paragraphs

In [147]:
%%time
article_id, latest_article_text, article_paragraphs, translated_paragraphs = get_and_translate_article_text(article_id, translate=True)

Reading the article at URL https://www.cranial-insertion.com/staff/articles/4376/edit ...
Using French language and my admin cookie...
The web page has been read and it's title is « <title>Cranial Insertion | Manage Articles</title> » !


0it [00:00, ?it/s]



0-th English paragraph (of length 93 letters and 12 words)


> [cright=Living Death]"Alex, the answer is "What would be an
example of an oximoron?"[/cright]

Translated 0-th paragraph in French (of length 87 letters and 10 words)


...



57-th English paragraph (of length 62 letters and 10 words)


> - Justin Hovdenes AKA Hovey
Level 2 Magic Judge
Rapid City, SD

Translated 57-th paragraph in French (of length 65 letters and 12 words)


> - Justin Hovdenes alias Hovey Niveau 2 Magic Judge Rapid City, SD

CPU times: user 41.9 s, sys: 108 ms, total: 42.1 s
Wall time: 43.3 s


40 seconds to generate these 57 first draft of translation, I think that's a great performance!

In [30]:
latest_article_link = f"https://www.cranial-insertion.com/article/{article_id}"
print("- The latest article is here:", latest_article_link)
print("- It was published on:", latest_article_published_data)
print("- The article ID is:", article_id)

- The latest article is here: https://www.cranial-insertion.com/article/4376
- It was published on: Mon, 31 Mar 2025 00:00:00 EDT
- The article ID is: 4376


In [31]:
print("- Its title in English is:")
print_paragraph(latest_article_title)
latest_article_title_french = en2fr(latest_article_title)
print("- A suggestion of a title in French is:")
print_paragraph(latest_article_title_french)

- Its title in English is:


> Cranial Insertion: All in Jeopardy (Can you answer in the form of a question?)

- A suggestion of a title in French is:


> Cranial Insertion: All in Jeopardy (Pouvez-vous répondre sous la forme d'une question?)

Let's now generate the content to write to the Google doc:

Google doc does **not** support Markdown markup, of course (it's more a clone of Microsoft Word), so the markup I use below is not rendered correctly by default. There is a hack below for that (for titles and lists, nothing more).

In [91]:
def generate_content_to_write(article_id, article_paragraphs, translated_paragraphs, translate=False):

    latest_article_link = f"https://www.cranial-insertion.com/article/{article_id}"

    entire_content_to_write = f"""
    # TITLE: « {latest_article_title} »

    - Une suggestion pour la traduction de ce titre en français est :
    TITRE : « {latest_article_title_french} »

    - Le dernier article vient de cette page web : {latest_article_link}
    - Il a été publié le : {latest_article_published_data}

    -----------------------------------------------------
    """

    for i in range(len(article_paragraphs)):
        paragraph = article_paragraphs[i]
        #print(f"\n\n{i}-th English paragraph: ")

        if translate:
            paragraph = paragraph.replace("\n", "\n    ")
            entire_content_to_write += f"\n\n    {paragraph}"
            translated_paragraph = translated_paragraphs[i]
            #print(f"Translated {i}-th paragraph in French: ")
            entire_content_to_write += f"\n\n{translated_paragraph}"
        else:
            entire_content_to_write += f"\n\n{paragraph}"

    return entire_content_to_write

We are now ready to us the previous functions:

In [ ]:
%%time

article_id = get_latest_article_id()

article_id, latest_article_text, article_paragraphs, translated_paragraphs = get_and_translate_article_text(article_id)

entire_content_to_write = generate_content_to_write(article_id, article_paragraphs, translated_paragraphs)

print("The entire content to write is:")
print_paragraph(entire_content_to_write)

---------

## Great! Now, let's upload this text automatically to a new tab in our Google doc!

First, I'll use a personnal Google doc, to test things out.

In [115]:
print_paragraph(entire_content_to_write[:1000])

> 
# TITLE: « Cranial Insertion: All in Jeopardy (Can you answer in the form of a question?) »

A suggestion of a translation of this title in French is:
TITRE : « Cranial Insertion: All in Jeopardy (Pouvez-vous répondre sous la forme d'une question?) »

- It was published on: Mon, 31 Mar 2025 00:00:00 EDT
- The article ID is: 4376
- The latest article is from this webpage:
    http://www.cranial-insertion.com/article/4376

-----------------------------------------------------


> [cright=Living Death]"Alex, the answer is "What would be an
example of an oximoron?"[/cright]

[crightLiving Death]"Alex, la réponse est "Quel serait un exemple d'oximoron?"[/cright]

> Today we have a collection of questions found on the IRC Chatroom #magicjudges-rules and the Facebook group Ask the Judge - [b]Magic[/b]: The Gathering Rules and Policy Questions. Feel free to join us in either or both groups.

Aujourd'hui, nous avons une collection de questions trouvées sur le IRC Chatroom magicjudges-rules et 

In [33]:
our_google_doc_url = "https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing"
print("The Google doc URL is:")
print(our_google_doc_url)

our_google_doc_id = our_google_doc_url.split("/")[-2]
print("The Google doc ID is:")
print(our_google_doc_id)

The Google doc URL is:
https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing
The Google doc ID is:
1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo


Let's try to write a text in a new tab of this Google doc.
It requires this code:

In [34]:
from googleapiclient.discovery import build
from google.colab import auth

auth.authenticate_user()

In [35]:
default_content_to_write_to_this_google_doc = """
# Title of level 1
## Title of level 2
### Title of level 3

*Hello, world!*
**WHO awesome!**

> Sent from a Google colab notebook running Python! Code by @Naereen

Using <https://colab.research.google.com/> for free (but maybe soon for Pro).
See [this page for more information](https://colab.research.google.com/signup).
"""

Thanks to <https://github.com/PragatiGunai1097/google-docs-markdown-formatter/blob/main/MarkdownMeeting_ToGoogleDoc_.ipynb>:

In [67]:
# Step 3: Parse markdown to structure content
def markdown_to_requests(md_content, index=0):
    lines = md_content.split("\n")
    requests = []
    index = index + 1

    for line in lines:
        if line.startswith("# "):  # Heading 1
            requests.append({"insertText": {"location": {"index": index}, "text": line[2:] + "\n"}})
            index += len(line[2:]) + 1
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index - len(line[2:]) - 1, "endIndex": index - 1},
                    "paragraphStyle": {
                        "namedStyleType": "HEADING_1",  # Use HEADING_1 for main heading
                    },
                    "fields": "namedStyleType",
                    #"textStyle": {"bold": True, "fontSize": {"magnitude": 24, #"unit": "PT"}},
                    #"fields": "bold,fontSize"
                }
            })
        elif line.startswith("## "):  # Heading 2
            requests.append({"insertText": {"location": {"index": index}, "text": line[3:] + "\n"}})
            index += len(line[3:]) + 1
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index - len(line[3:]) - 1, "endIndex": index - 1},
                    "paragraphStyle": {
                        "namedStyleType": "HEADING_2",  # Use HEADING_2 for main heading
                    },
                    "fields": "namedStyleType",
                    #"textStyle": {"bold": True, "fontSize": {"magnitude": 18, "unit": "PT"}},
                    #"fields": "bold,fontSize"
                }
            })
        elif line.startswith("### "):  # Heading 3
            requests.append({"insertText": {"location": {"index": index}, "text": line[4:] + "\n"}})
            index += len(line[4:]) + 1
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index - len(line[4:]) - 1, "endIndex": index - 1},
                    "paragraphStyle": {
                        "namedStyleType": "HEADING_3",  # Use HEADING_3 for main heading
                    },
                    "fields": "namedStyleType",
                    #"textStyle": {"bold": True, "fontSize": {"magnitude": 16, "unit": "PT"}},
                    #"fields": "bold,fontSize"
                }
            })
        elif line.startswith("#### "):  # Heading 4
            requests.append({"insertText": {"location": {"index": index}, "text": line[5:] + "\n"}})
            index += len(line[5:]) + 1
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index - len(line[5:]) - 1, "endIndex": index - 1},
                    "paragraphStyle": {
                        "namedStyleType": "HEADING_4",  # Use HEADING_4 for main heading
                    },
                    "fields": "namedStyleType",
                    #"textStyle": {"bold": True, "fontSize": {"magnitude": 16, "unit": "PT"}},
                    #"fields": "bold,fontSize"
                }
            })
        elif line.startswith("- [ ]"):  # Checkboxes
            requests.append({"insertText": {"location": {"index": index}, "text": "☐ " + line[6:] + "\n"}})
            index += len("☐ " + line[6:]) + 1
        elif line.startswith("- "):  # Bullet points
            requests.append({"insertText": {"location": {"index": index}, "text": "• " + line[2:] + "\n"}})
            index += len("• " + line[2:]) + 1
        elif line.strip():  # Normal text
            requests.append({"insertText": {"location": {"index": index}, "text": line + "\n"}})
            index += len(line) + 1
        else:  # Blank lines
            requests.append({"insertText": {"location": {"index": index}, "text": "\n"}})
            index += 1

    return requests

We can use a simple request, or the more-complex request from above:

In [68]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/documents']
# The ID of the document to insert text into.
DOCUMENT_ID = '1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo'

def write_content_to_google_doc(content_to_write=default_content_to_write_to_this_google_doc, markdown=False):
    service = build('docs', 'v1', credentials=None)

    # Create a new tab/section in the document.
    requests = [
        {'insertText':
            {
                'location': {'index': 1},
                'text':
                    content_to_write
            }
        },
    ]

    # Convert the markdown content into Google Docs API requests
    if markdown:
        requests = markdown_to_requests(content_to_write)

    result = service.documents().batchUpdate(documentId=DOCUMENT_ID, body={'requests': requests}).execute()
    print(result)

    print(f"The Google doc at this URL has been modified with the desired content of length {len(content_to_write)}:")
    print(our_google_doc_url)

I wanted to be able to create a new tab, but apparently this can't be done using the current version of Google Docs API.

In [85]:
# prompt: Create a new tab in this Google document, using the Google Docs API requests
def create_new_tab_in_google_doc(content, new_tab_name="New tab", markdown=False, document_id=DOCUMENT_ID):
    """Creates a new tab/section in a Google Doc and inserts content.

    Args:
        content: The content to insert into the new tab.
        document_id: The ID of the Google Doc.
    """
    try:
        service = build('docs', 'v1', credentials=None)
        # Find the current document's last index
        document = service.documents().get(documentId=document_id).execute()
        last_index = document['body']['content'][-1]['endIndex']
        print(f"last_index = {last_index} ...")
        last_index = 1
        # Create a new section at the end
        requests = [
            {
                'insertSectionBreak': {
                    'location': {'index': last_index},
                    'sectionType': 'NEXT_PAGE'  # Or SECTION_BREAK
                }
            }
        ]

        # Set the name of the section
        requests += [
            {
                "updateSectionStyle": {
                    "sectionStyle": {
                        "sectionType": "NEXT_PAGE",
                        #"info": {
                        #    "sectionId": new_tab_name, # new_tab_name is used as sectionId
                        #    "displayName": new_tab_name
                        #}
                    },
                    "range": {
                        "startIndex": last_index + 1,
                        "endIndex": last_index + 2,
                    },
                    #"fields": "info.displayName,info.sectionId"
                    "fields": "*"
                }
            }
        ]

        # Write the text as a basic index in the current tab
        if not markdown:
            requests += [
                {'insertText': {
                    'location': {'index': last_index + 1},
                    'text': content
                }}
            ]
        else:
            # Convert the markdown content into Google Docs API requests
            requests += markdown_to_requests(content, index=last_index+2)

        # Execute the requests
        result = service.documents().batchUpdate(
            documentId=document_id, body={'requests': requests}).execute()
        print(f"Successfully created a new tab in the Google Doc: {result}")
    except Exception as e:
        print(f"An error occurred: {e}")
        raise e

We can now test it! It should support basic formatting using Markdown.

In [78]:
%%time
write_content_to_google_doc(default_content_to_write_to_this_google_doc, markdown=True)

{'replies': [{}], 'writeControl': {'requiredRevisionId': 'ALBJ4LvFW-EtzfvML7O3Dya5CHgs_dmY-s7wXlijx-8_A4cljaaBq2bl_-lOzZGfNrJEhqxb_1hn2mEH5Ldj-w'}, 'documentId': '1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo'}
The Google doc at this URL has been modified with the desired content of length 324:
https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing
CPU times: user 188 ms, sys: 9.95 ms, total: 198 ms
Wall time: 738 ms


I tried for a long time different requests to add a new tab, but I'm pretty sure it's not supported by the API.

In [90]:
%%time
if False:
    # Example usage (replace with your actual content)
    new_tab_content = "This is the content of the new tab."

    create_new_tab_in_google_doc(new_tab_content, new_tab_name="Nouvel onglet")

last_index = 2 ...
An error occurred: <HttpError 400 when requesting https://docs.googleapis.com/v1/documents/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo:batchUpdate?alt=json returned "Invalid requests[1].updateSectionStyle: A value for top margin must be specified in order to update it.". Details: "Invalid requests[1].updateSectionStyle: A value for top margin must be specified in order to update it.">


HttpError: <HttpError 400 when requesting https://docs.googleapis.com/v1/documents/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo:batchUpdate?alt=json returned "Invalid requests[1].updateSectionStyle: A value for top margin must be specified in order to update it.". Details: "Invalid requests[1].updateSectionStyle: A value for top margin must be specified in order to update it.">

Now, if adding this first line worked, let's update the Google doc with all the paragraphs and their draft of translation.

In [92]:
def get_article_translate_it_then_upload_to_google_doc(article_id=None, translate=False):
    if article_id is None:
        article_id = get_latest_article_id()
    print(f"- The latest article ID is: {article_id}... Downloading it, then translating it...")

    article_id, latest_article_text, article_paragraphs, translated_paragraphs = get_and_translate_article_text(article_id, translate=translate)

    print("- Generating the Markdown content to write to Google doc:")
    entire_content_to_write = generate_content_to_write(article_id, article_paragraphs, translated_paragraphs, translate=translate)

    print("- The entire content to write is:")
    print_paragraph(entire_content_to_write)

    return write_content_to_google_doc(entire_content_to_write, markdown=True)

In [95]:
%%time
get_article_translate_it_then_upload_to_google_doc(article_id=get_latest_article_id(), translate=False)

- The latest article ID is: 4376... Downloading it, then translating it...
Reading the article at URL https://www.cranial-insertion.com/staff/articles/4376/edit ...
Using French language and my admin cookie...
The web page has been read and it's title is « <title>Cranial Insertion | Manage Articles</title> » !


0it [00:00, ?it/s]



0-th English paragraph (of length 93 letters and 12 words)


> [cright=Living Death]"Alex, the answer is "What would be an
example of an oximoron?"[/cright]



1-th English paragraph (of length 226 letters and 38 words)


...



57-th English paragraph (of length 62 letters and 10 words)


> - Justin Hovdenes AKA Hovey
Level 2 Magic Judge
Rapid City, SD

- Generating the Markdown content to write to Google doc:
The entire content to write is:


> 
    # TITLE: « Cranial Insertion: All in Jeopardy (Can you answer in the form of a question?) »

    - Une suggestion pour la traduction de ce titre en français est :
    TITRE : « Cranial Insertion: All in Jeopardy (Pouvez-vous répondre sous la forme d'une question?) »

    - Le dernier article vient de cette page web : https://www.cranial-insertion.com/article/4376
    - Il a été publié le : Mon, 31 Mar 2025 00:00:00 EDT

    -----------------------------------------------------
    

[cright=Living Death]"Alex, the answer is "What would be an
example of an oximoron?"[/cright]

Today we have a collection of questions found on the IRC Chatroom #magicjudges-rules and the Facebook group Ask the Judge - [b]Magic[/b]: The Gathering Rules and Policy Questions. Feel free to join us in either or both groups.

If you have any [b]Magic[/b] questions burning at the back of your brain, you can send them to us. We may even use them in a future article. If you have a short question, you can send it to us via our Twitter account at [url=https://twitter.com/CranialTweet]@CranialTweet[/url], and you can send us longer questions at [email]moko@cranialinsertion.com[/email].

[hr]
[Q] I have two [c]Squire[/c]s on the battlefield and I have [c]Custodi Soulbinders[/c] and four copies of [c]Bear Cub[/c] in my graveyard. If I cast [c]Living Death[/c], how many +1/+1 counters will Soulbinders enter with?[/Q]

[A] Custodi Soulbinders will enter with zero +1/+1 counters. Souldbinders and the Bears will enter simultaneously. Soulbinders' first ability is a replacement effect that checks to see how many creatures are on the battlefield just before it enters. At that point there are no creatures on the board, so it gets no counters as it enters.[/A]

[hr]
[Q] My commander is [c]Grimgrin, Corpse-Born[/c]. Grimgrin was [c]Murder[/c]ed last turn and I choose to move him to the command zone. I control [c]Rooftop Storm[/c]. If I try to cast Grimgrin from the command zone, will he cost [m]2[/m] or will he be free?[/Q]

[A] He will cost [m]2[/m]. 
Commander "tax" is an additional cost that must be paid in addition to the normal cost for casting your commander. Rooftop Storm gives you the option of replacing the normal cost of casting your Zombies with [m]0[/m]. This has no effect on additional costs.  [/A]

[hr]
[Q] My opponent exiled [c]Wheel of Fate[/c] with [c]Reckless Impulse[/c]. She now claims she can cast Wheel of Fate because it's in exile. Is that true?[/Q]

[A] Your opponent is wrong. The part of suspend that lets you cast a card with no mana cost is casting it without paying its mana cost, not that it's casting it from exile.[/A]

[hr]
[Q] I control four [c]Bear Cub[/c]s, [c]Agrus Kos, Eternal Soldier[/c], and [c]Thundering Raiju[/c]. None of my creatures are currently modified. I attack with the Raiju and have its attack trigger target Agrus. When Agrus's ability triggers because he became targeted, I pay the cost. After all these triggers resolve, how much damage will Raiju deal to each opponent?[/Q]

[A] The Answer is between 15-19 damage, depending on when you let the trigger targeting Raiju resolve.

When Agrus's trigger resolves, it creates five copies of Raiju's trigger, one for each Bear and one for Raiju, making a total of six Raiju triggers. The last trigger to resolve has to be the one targeting Agrus, and the other five resolve before it in the order of your choosing. 

For our first example, we will have the trigger targeting Raiju resolve first, then the four triggers targeting Bears, and finally the one targeting Agrus.

Raiju gets a +1/+1 counter, no damage is dealt as Raiju only checks modified creatures other than himself. Total damage 0.
Bear #1 gets a +1/+1 counter, 1 damage is dealt as there is only one modified creature other than Raiju. Total damage 1.
Bear #2 gets a +1/+1 counter, 2 damage is dealt as there are now two modified creatures. Total damage 3.
Bear #3 gets a +1/+1 counter, 3 damage is dealt. Total damage 6.
Bear #4 gets a counter, 4 damage is dealt. Total damage of 10.
Agrus get a a counter, 5 damage is dealt. Total damage of 15.

Now let's have the four Bears' triggers resolve first, then Raiju, followed by Agrus.
Bear #1, 1 damage is dealt. Total damage 1.
Bear #2, 2 damage is dealt. Total damage 3.
Bear #3, 3 damage id dealt. Total damage 6.
Bear #4, 4 damage is dealt. Total damage is 10.
Raiju, 4 damage is dealt. Total damage is 14.
Agrus, 5 damage is dealt. Total damage is 19.

The later the trigger targeting Raiju resolves, the more the total damage.
 [/A]

[hr]
[cleft=Grafdigger's Cage]Alex, the answer is "What is used to 
keep the dead from escaping their graves?"[/cleft][Q] Does [c]Grafdigger's Cage[/c] stop [c]Abhorrent Oculus[/c] from manifest-dreading?[/Q]

[A] Well, in the important ways, yes. They still "manifest dread", but some parts they can't complete. Manifest Dread is "Look at the top two cards of your library. Manifest one of them, then put the cards you looked at that were not manifested this way into your graveyard." Since you can't put a creature from your library into play, both cards looked at are put into the graveyard.[/A]

[hr]
[Q] Last turn my opponent stole my [c]Bear Cub[/c] by casting [c]Sower of Temptation[/c]. This turn I used [c]Ovinize[/c] to make the Sower lose all its abilities. I did this because I think I'll get my Bear Cub back, since the Sower no longer has the "steal your creature" ability. My opponent disagrees. Who is right?[/Q]

[A] Your opponent is correct. Removing the enters ability (that creates an effect with a duration) after the ability has resolved, doesn't stop it from functioning.

If it helps, an example that seems more intuitive would be the following: My opponent used [c]Rootwater Matriarch[/c] to steel my [c]Hill Giant[/c] enchanted with [c]Spirit Link[/c]. Killing the Matriarch won't get the Hill Giant back, even though that removes where the steal ability came from. The duration is based on whether or not the Hill Giant is still enchanted.

So with the original question, removing Sower's abilities doesn't stop the steal effect. You have to remove the Sower to gain the creature back because that ends the duration of the effect.[/A]

[hr]
[Q] So continuing with the last question, what if instead of [c]Ovinize[/c]ing the [c]Sower of Temptation[/c] I phased it out with [c]Reality Ripple[/c]?[/Q]

[A] You would get your creature back. If a creature phases out, it's treated as though it's no longer on the battlefield. The duration of its effect has been met, so the effect ends and the stolen creature returns to its last controller, usually its owner.[/A]

[hr]
[Q] I block my opponent's attacking [c]Bear Cub[/c] with [c]Manor Gargoyle[/c]. After combat ends, the opponent then casts [c]Disfigure[/c] targeting my Gargoyle. They claim they just killed my Gargoyle. Is this true?[/Q]

[A] It is not true, your Gargoyle lives through this. Many new players make this same wrong assumption. Damage doesn't reduce toughness. You have a 2/2 Gargoyle with 2 damage marked on it. Because it's indestructible, the state based action that tries to destroy it for having equal or more damage than it has toughness fails to do anything. If they want to get your Gargoyle into the graveyard, they need to give it enough [c=Dismember]-X/-X to make its toughness 0 or less[/c], or force you to [c=Diabolic Edict]sacrifice it[/c].[/A]

[hr]
[Q] If [c]Ragavan, Nimble Pilferer[/c] is my commander, can I cast him for his dash cost from the command zone?[/Q]

[A] Yes, you can. Dash can be used from any zone, so long as you are allowed to cast that card from that same zone normally. [/A]

[hr]
[Q] Following up with the last question, do I have to pay commander tax if I dash out my commander?[/Q]

[A] Yes, you do. Suppose you cast Ragavan normally on turn one, then later he died and returned to the command zone. When you go to dash him he now costs [m]3[/m][m]R[/m] to dash. Then if he dies again and returns to the command zone, the next time you go to cast him he costs either [m]4[/m][m]R[/m] to cast normally or [m]5[/m][m]R[/m] to dash.[/A]

[hr]
[Q] I have eight cards in my hand. I go to my cleanup step and discard [c]Blazing Rootwalla[/c] to get down to maximum hand size. Can I cast the Rootwalla even though I'm in my cleanup step?[/Q]

[A] Yes, you can. During most cleanup steps, no players get priority. However, if an ability triggers during the cleanup step, or a state based action applies during a cleanup step, then the active player gains priority and may cast spells and activate abilities. 

This is when you can cast the Rootwalla, as discarding it to madness puts a trigger on the stack.

A normal priority pass will begin from here and until all players pass in succession with an empty stack, players may add new items to the stack when they have priority. Once all players pass with an empty stack, this cleanup step ends and a new one begins. If that new cleanup ends normally, then the game will move on to the next player's turn.[/A]

[hr]
[Q] While resolving [c]Burning Inquiry[/c] I discarded [c]Basking Rootwalla[/c] and two [c]Mountain[/c]s. My opponent controls a [c]Rest in Peace[/c]. My opponent claims I can't cast the Rootwalla for its Madness cost as it got exiled by Rest in Peace when I discarded it. Is this true?[/Q]

[A] Your opponent is wrong. You can cast your Rootwalla for its madness cost. The Rootwalla hasn't been exiled by Rest in Peace yet. This is because madness doesn't quite work the way most people think it does. If a card with madness gets discarded, it goes to exile instead of being put into its owner's graveyard. Then the next time a player would get priority, a trigger allowing its owner to cast it for its madness cost is put on the stack. When this trigger resolves, the owner may cast the card, but if they don't then the card is put into their graveyard. 

To get back to the original question, Rest in Peace can only exile a discarded Madness card after the owner has decided not to cast it for madness and choses to let it go to the graveyard.
[/A]

[hr]
[Q] My opponent pays [m]3[/m][m]U[/m] to gain level 2 on their [c]Stormchaser's Talent[/c]. Before it becomes level 2, I [c]Boomerang[/c] it to their hand. The opponent says they still get to return an instant or sorcery to their hand, I say they don't, who is right?[/Q]

[A] You are correct, they do not. Gaining a level is an ability that uses the stack and can be responded to. Stormchaser's level 2 ability is a trigger that triggers upon it becoming level 2 and happens only after it becomes level 2. If it's not on the battlefield when its level 2 ability resolves, there is no ability to trigger (nor a permanent to become level 2). [/A]

[hr]
[cright=Bear Cub|PO2]Alex, the answer is "What is another
name for Ursine Child?"[/cright][Q] I enchanted my opponent's [c]Bear Cub[/c] with my [c]Sigil of Sleep[/c]. If the Bear Cub goes unblocked attacking me, who chooses which creature I control gets returned to its owner's hand?[/Q]

[A] You get to choose as you control the Sigil of Sleep.[/A]

[hr]
[Q] I attack with [c]Geist of Saint Traft[/c] while I control [c]Anointed Procession[/c]. When combat ends, will both Angel tokens get exiled or do I get to keep one?[/Q]

[A] Both still get exiled. Saint Traft's triggered ability still made both tokens and tracks them to be exiled at the end of combat. Procession doesn't make tokens, it just changes how many tokens other cards and abilities make.[/A]

[hr]
[Q] I attack with [c]Geist of Saint Traft[/c], but before blocks, I cast [c]Eyes in the Skies[/c] choosing to populate the Angel token. When combat ends, will both Angel tokens get exiled or do I get to keep one?[/Q]

[A] You get to keep the one made by Eyes in the Skies. Unlike Anointed Procession from the last question, Eyes in the Skies makes the token and the token has no link to the ability that made the original Angel token. This includes that the second Angel token is not attacking, is not tapped, and it won't get exiled at the end of combat.[/A]

[hr]
[Q] I don't understand what [c]Emrakul, the World Anew[/c] has protection from. Can you clarify this?[/Q]

[A] Emrakul has protection from spells and separately protection from permanents cast "this turn". This turn is whichever turn it currently is, even if it's been many turns since Emrakul herself was cast. 

So for example, if I cast [c]Visara the Dreadful[/c] and gave her some [c]Lightning Greaves[/c] the same turn I cast her, she can't target and destroy Emrakul that turn. However, if I pass to my opponent's turn, Visara can now kill Emrakul as it's no longer the same turn Visara was cast. If I used [c]Goryo's Vengeance[/c] to reanimate Visara, then Visara could kill Emrakul that same turn as Visara wasn't cast at all.

Emrakul's protection ability acts similar to having protection from Auras. Auras when cast target what they will enchant, so Emrakul is safe from most Auras. However, on later turns, an Aura could be moved from its original target, to enchant Emrakul. For example, you could move [c]Pacifism[/c] from a 1/1 Goblin token to Emrakul using [c]Crown of the Ages[/c]. 
[/A]

[hr]
[Q] I cast [c]Season of Weaving[/c] choosing the two paws mode (to copy my [c]Arcanis the Omnipotent[/c]) and the three paws mode. Will I have to put one Arcanis into my graveyard due to the legend rule before bouncing all nonland and nontoken permanents?[/Q]

[A] You will not and you can not. The legend rule is a state based action. It's never checked in the middle of resolving a spell. You will put the original Arcanis into your hand before Season of Weaving fully resolves. The legend rule doesn't come into play during this situation.[/A]

[hr]
[Q] If I cast [c]Righteous Confluence[/c] choosing mode two once to exile my opponent's [c]Leyline of Sanctity[/c] and mode three twice to gain 10 total life, and the opponent responds by [c]Boomerang[/c]ing the Leyline back to their hand, do I still gain 10 life?[/Q]

[A] You do not. Righteous Confluence had only one target, Leyline of Sanctity, and that target has disappeared. If a spell loses all of its targets by the time it goes to resolve, it fizzles and none of its effects happen. [/A]

[hr]
[Q] I just cast [c]Cultivate[/c] putting a [c]Swamp[/c] onto the battlefield and a [c]Forest[/c] into my hand. I haven't played a land from my hand yet this turn. Can I still play the Forest as my land drop for the turn, or did the Swamp use up my land drop for the turn?[/Q]

[A] You may still play the Forest. Putting a land onto the battlefield with a spell or ability doesn't use up your one land drop per turn.[/A]

[hr]

Each Question in my last article had a hidden reference to a different pop-culture pig. With ten different questions there were ten different pigs. The spoiler below has the answer key to those references.
[spoiler]
Question 1) Charlotte's Web - Wilbur the Pig and Charlotte the Spider
Question 2) Jim Henson's Muppets - Miss Piggy, Kermit the Frog, and Fozzie Bear.
Question 3) The Three Little Pigs and the Big Bad Wolf
Question 4) Angry Birds Video Game (Birds vs Pigs)
Question 5) Looney Tunes - Bugs Bunny, Porky Pig, and Daffy Duck
Question 6) Babe the 1995 Movie - Babe the Pig, Sheep for Babe to herd, and Farmer Arthur Hoggett
Question 7) Legend of Zelda - Link an Elf Warrior, Zelda an Elf Noble, and Ganon (pig form of Ganondorf)
Question 8) The Lion King - Timon the Meerkat, Pumbaa the Warthog, and some Hyenas for them to fight.
Question 9) Toy Story - Woody the Cowboy, Buzz the Spaceman, Rex the T-Rex, and Hamm the Piggy Bank.
Question 10) Winnie the Pooh - Pooh the Bear, Tigger the Tiger, and Piglet the Pig.[/spoiler]

- Justin Hovdenes AKA Hovey
Level 2 Magic Judge
Rapid City, SD

{'replies': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}], 'writeControl': {'requiredRevisionId': 'ALBJ4Lvf-aR1vWR3U4DLEgKLHARlbWPKDeFG9Gik11P3M39mcj5fxPa4nWODJwU3SaQ7NsirbZUjnVTEcvEyHQ'}, 'documentId': '1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo'}
The Google doc at this URL has been modified with the desired content of length 15

----------

## Automatically send a short message to our Discord now that our Google doc has been set up for this weekend's translation job

First, I'll use my own private Discord server to try this out.
Then if everything works, I'll use our Discord server.

I'm following [this article](https://www.makerhacks.com/python-messages-discord/), after I created a webhook on my server by following [the official tutorial](https://support.discord.com/hc/en-us/articles/228383668-Intro-to-Webhooks).

In [96]:
# web library
import http.client

In [97]:
# your webhook URL
my_webhook_url = """https://discord.com/api/webhooks/1358504126408818940/0gRa5xMe1d8-JxImSqsHFeZUu0EK338RD5sa0GedxqFtGp8DHAEfqHvltnRRbfFl7F_x"""

def send_message_to_Discord(message, webhookurl=my_webhook_url):
    print(f"Starting to send the following message to my Discord server:\n{message}")

    # compile the form data (BOUNDARY can be anything)
    formdata = u"------:::BOUNDARY:::\r\nContent-Disposition: form-data; name=\"content\"\r\n\r\n" + message + u"\r\n------:::BOUNDARY:::--"

    # See https://stackoverflow.com/questions/74722329/how-to-send-accented-characters-with-diacritics-in-http-request-payload#74722361
    # formdata = formdata.encode('iso-8859-1')
    formdata = formdata.encode('utf-8')

    # get the connection and make the request
    connection = http.client.HTTPSConnection("discord.com")
    connection.request("POST", webhookurl, formdata, {
        'content-type': "multipart/form-data; boundary=----:::BOUNDARY:::",
        'cache-control': "no-cache",
        # 'encoding': 'utf-8'
    })

    # get the response
    response = connection.getresponse()
    result = response.read()

    # return back to the calling function with the result
    result_decoded_in_utf8 = result.decode("utf-8")
    print("Success!")
    if result_decoded_in_utf8:
        print(result_decoded_in_utf8)
    return result_decoded_in_utf8

In [98]:
send_message_to_Discord(u"*Hello, world!*\n> Sent from a Google colab notebook running Python! Code by Naereen")

Starting to send the following message to my Discord server:
*Hello, world!*
> Sent from a Google colab notebook running Python! Code by Naereen
Success!


''

--------------------------

## Do it all in one function: download article, translate it (if running on GPU), upload to GDoc, notify the team, make coffee!

In [99]:
def do_all_then_notifyme(article_id=None, translate=False):
    if article_id is None:
        article_id = get_latest_article_id()
    print(f"- The latest article ID is: {article_id}... Downloading it...")
    if translate:
        print("Then translating it...")
    print("Then uploading it to our Google doc...")

    get_article_translate_it_then_upload_to_google_doc(article_id, translate=translate)

    latest_article_link = f"https://www.cranial-insertion.com/article/{article_id}"

    message_for_Discord = f"""||@everyone|| [Notre Google doc partagé]({our_google_doc_url}) a été mis à jour, avec le brouillon de traduction pour [le nouvel article hebdomadaire]({latest_article_link}).

Cet article s'intitule "{latest_article_title}", a été publié le {latest_article_published_data}, et son ID est {article_id}.

> Envoyé automatiquement depuis un notebook Python executé sur Google Colab! Code by @Naereen
"""
    print_paragraph(message_for_Discord)

    return send_message_to_Discord(message_for_Discord)

Let's try!

In [102]:
%%time
do_all_then_notifyme(translate=running_on_GPU)

- The latest article ID is: 4376... Downloading it...
Then uploading it to our Google doc...
- The latest article ID is: 4376... Downloading it, then translating it...
Reading the article at URL https://www.cranial-insertion.com/staff/articles/4376/edit ...
Using French language and my admin cookie...
The web page has been read and it's title is « <title>Cranial Insertion | Manage Articles</title> » !


0it [00:00, ?it/s]



0-th English paragraph (of length 93 letters and 12 words)


> [cright=Living Death]"Alex, the answer is "What would be an
example of an oximoron?"[/cright]



1-th English paragraph (of length 226 letters and 38 words)


...

> 
    # TITLE: « Cranial Insertion: All in Jeopardy (Can you answer in the form of a question?) »

    - Une suggestion pour la traduction de ce titre en français est :
    TITRE : « Cranial Insertion: All in Jeopardy (Pouvez-vous répondre sous la forme d'une question?) »

    - Le dernier article vient de cette page web : https://www.cranial-insertion.com/article/4376
    - Il a été publié le : Mon, 31 Mar 2025 00:00:00 EDT

    -----------------------------------------------------
    

[cright=Living Death]"Alex, the answer is "What would be an
example of an oximoron?"[/cright]

Today we have a collection of questions found on the IRC Chatroom #magicjudges-rules and the Facebook group Ask the Judge - [b]Magic[/b]: The Gathering Rules and Policy Questions. Feel free to join us in either or both groups.

If you have any [b]Magic[/b] questions burning at the back of your brain, you can send them to us. We may even use them in a future article. If you have a short question, you can send it to us via our Twitter account at [url=https://twitter.com/CranialTweet]@CranialTweet[/url], and you can send us longer questions at [email]moko@cranialinsertion.com[/email].

[hr]
[Q] I have two [c]Squire[/c]s on the battlefield and I have [c]Custodi Soulbinders[/c] and four copies of [c]Bear Cub[/c] in my graveyard. If I cast [c]Living Death[/c], how many +1/+1 counters will Soulbinders enter with?[/Q]

[A] Custodi Soulbinders will enter with zero +1/+1 counters. Souldbinders and the Bears will enter simultaneously. Soulbinders' first ability is a replacement effect that checks to see how many creatures are on the battlefield just before it enters. At that point there are no creatures on the board, so it gets no counters as it enters.[/A]

[hr]
[Q] My commander is [c]Grimgrin, Corpse-Born[/c]. Grimgrin was [c]Murder[/c]ed last turn and I choose to move him to the command zone. I control [c]Rooftop Storm[/c]. If I try to cast Grimgrin from the command zone, will he cost [m]2[/m] or will he be free?[/Q]

[A] He will cost [m]2[/m]. 
Commander "tax" is an additional cost that must be paid in addition to the normal cost for casting your commander. Rooftop Storm gives you the option of replacing the normal cost of casting your Zombies with [m]0[/m]. This has no effect on additional costs.  [/A]

[hr]
[Q] My opponent exiled [c]Wheel of Fate[/c] with [c]Reckless Impulse[/c]. She now claims she can cast Wheel of Fate because it's in exile. Is that true?[/Q]

[A] Your opponent is wrong. The part of suspend that lets you cast a card with no mana cost is casting it without paying its mana cost, not that it's casting it from exile.[/A]

[hr]
[Q] I control four [c]Bear Cub[/c]s, [c]Agrus Kos, Eternal Soldier[/c], and [c]Thundering Raiju[/c]. None of my creatures are currently modified. I attack with the Raiju and have its attack trigger target Agrus. When Agrus's ability triggers because he became targeted, I pay the cost. After all these triggers resolve, how much damage will Raiju deal to each opponent?[/Q]

[A] The Answer is between 15-19 damage, depending on when you let the trigger targeting Raiju resolve.

When Agrus's trigger resolves, it creates five copies of Raiju's trigger, one for each Bear and one for Raiju, making a total of six Raiju triggers. The last trigger to resolve has to be the one targeting Agrus, and the other five resolve before it in the order of your choosing. 

For our first example, we will have the trigger targeting Raiju resolve first, then the four triggers targeting Bears, and finally the one targeting Agrus.

Raiju gets a +1/+1 counter, no damage is dealt as Raiju only checks modified creatures other than himself. Total damage 0.
Bear #1 gets a +1/+1 counter, 1 damage is dealt as there is only one modified creature other than Raiju. Total damage 1.
Bear #2 gets a +1/+1 counter, 2 damage is dealt as there are now two modified creatures. Total damage 3.
Bear #3 gets a +1/+1 counter, 3 damage is dealt. Total damage 6.
Bear #4 gets a counter, 4 damage is dealt. Total damage of 10.
Agrus get a a counter, 5 damage is dealt. Total damage of 15.

Now let's have the four Bears' triggers resolve first, then Raiju, followed by Agrus.
Bear #1, 1 damage is dealt. Total damage 1.
Bear #2, 2 damage is dealt. Total damage 3.
Bear #3, 3 damage id dealt. Total damage 6.
Bear #4, 4 damage is dealt. Total damage is 10.
Raiju, 4 damage is dealt. Total damage is 14.
Agrus, 5 damage is dealt. Total damage is 19.

The later the trigger targeting Raiju resolves, the more the total damage.
 [/A]

[hr]
[cleft=Grafdigger's Cage]Alex, the answer is "What is used to 
keep the dead from escaping their graves?"[/cleft][Q] Does [c]Grafdigger's Cage[/c] stop [c]Abhorrent Oculus[/c] from manifest-dreading?[/Q]

[A] Well, in the important ways, yes. They still "manifest dread", but some parts they can't complete. Manifest Dread is "Look at the top two cards of your library. Manifest one of them, then put the cards you looked at that were not manifested this way into your graveyard." Since you can't put a creature from your library into play, both cards looked at are put into the graveyard.[/A]

[hr]
[Q] Last turn my opponent stole my [c]Bear Cub[/c] by casting [c]Sower of Temptation[/c]. This turn I used [c]Ovinize[/c] to make the Sower lose all its abilities. I did this because I think I'll get my Bear Cub back, since the Sower no longer has the "steal your creature" ability. My opponent disagrees. Who is right?[/Q]

[A] Your opponent is correct. Removing the enters ability (that creates an effect with a duration) after the ability has resolved, doesn't stop it from functioning.

If it helps, an example that seems more intuitive would be the following: My opponent used [c]Rootwater Matriarch[/c] to steel my [c]Hill Giant[/c] enchanted with [c]Spirit Link[/c]. Killing the Matriarch won't get the Hill Giant back, even though that removes where the steal ability came from. The duration is based on whether or not the Hill Giant is still enchanted.

So with the original question, removing Sower's abilities doesn't stop the steal effect. You have to remove the Sower to gain the creature back because that ends the duration of the effect.[/A]

[hr]
[Q] So continuing with the last question, what if instead of [c]Ovinize[/c]ing the [c]Sower of Temptation[/c] I phased it out with [c]Reality Ripple[/c]?[/Q]

[A] You would get your creature back. If a creature phases out, it's treated as though it's no longer on the battlefield. The duration of its effect has been met, so the effect ends and the stolen creature returns to its last controller, usually its owner.[/A]

[hr]
[Q] I block my opponent's attacking [c]Bear Cub[/c] with [c]Manor Gargoyle[/c]. After combat ends, the opponent then casts [c]Disfigure[/c] targeting my Gargoyle. They claim they just killed my Gargoyle. Is this true?[/Q]

[A] It is not true, your Gargoyle lives through this. Many new players make this same wrong assumption. Damage doesn't reduce toughness. You have a 2/2 Gargoyle with 2 damage marked on it. Because it's indestructible, the state based action that tries to destroy it for having equal or more damage than it has toughness fails to do anything. If they want to get your Gargoyle into the graveyard, they need to give it enough [c=Dismember]-X/-X to make its toughness 0 or less[/c], or force you to [c=Diabolic Edict]sacrifice it[/c].[/A]

[hr]
[Q] If [c]Ragavan, Nimble Pilferer[/c] is my commander, can I cast him for his dash cost from the command zone?[/Q]

[A] Yes, you can. Dash can be used from any zone, so long as you are allowed to cast that card from that same zone normally. [/A]

[hr]
[Q] Following up with the last question, do I have to pay commander tax if I dash out my commander?[/Q]

[A] Yes, you do. Suppose you cast Ragavan normally on turn one, then later he died and returned to the command zone. When you go to dash him he now costs [m]3[/m][m]R[/m] to dash. Then if he dies again and returns to the command zone, the next time you go to cast him he costs either [m]4[/m][m]R[/m] to cast normally or [m]5[/m][m]R[/m] to dash.[/A]

[hr]
[Q] I have eight cards in my hand. I go to my cleanup step and discard [c]Blazing Rootwalla[/c] to get down to maximum hand size. Can I cast the Rootwalla even though I'm in my cleanup step?[/Q]

[A] Yes, you can. During most cleanup steps, no players get priority. However, if an ability triggers during the cleanup step, or a state based action applies during a cleanup step, then the active player gains priority and may cast spells and activate abilities. 

This is when you can cast the Rootwalla, as discarding it to madness puts a trigger on the stack.

A normal priority pass will begin from here and until all players pass in succession with an empty stack, players may add new items to the stack when they have priority. Once all players pass with an empty stack, this cleanup step ends and a new one begins. If that new cleanup ends normally, then the game will move on to the next player's turn.[/A]

[hr]
[Q] While resolving [c]Burning Inquiry[/c] I discarded [c]Basking Rootwalla[/c] and two [c]Mountain[/c]s. My opponent controls a [c]Rest in Peace[/c]. My opponent claims I can't cast the Rootwalla for its Madness cost as it got exiled by Rest in Peace when I discarded it. Is this true?[/Q]

[A] Your opponent is wrong. You can cast your Rootwalla for its madness cost. The Rootwalla hasn't been exiled by Rest in Peace yet. This is because madness doesn't quite work the way most people think it does. If a card with madness gets discarded, it goes to exile instead of being put into its owner's graveyard. Then the next time a player would get priority, a trigger allowing its owner to cast it for its madness cost is put on the stack. When this trigger resolves, the owner may cast the card, but if they don't then the card is put into their graveyard. 

To get back to the original question, Rest in Peace can only exile a discarded Madness card after the owner has decided not to cast it for madness and choses to let it go to the graveyard.
[/A]

[hr]
[Q] My opponent pays [m]3[/m][m]U[/m] to gain level 2 on their [c]Stormchaser's Talent[/c]. Before it becomes level 2, I [c]Boomerang[/c] it to their hand. The opponent says they still get to return an instant or sorcery to their hand, I say they don't, who is right?[/Q]

[A] You are correct, they do not. Gaining a level is an ability that uses the stack and can be responded to. Stormchaser's level 2 ability is a trigger that triggers upon it becoming level 2 and happens only after it becomes level 2. If it's not on the battlefield when its level 2 ability resolves, there is no ability to trigger (nor a permanent to become level 2). [/A]

[hr]
[cright=Bear Cub|PO2]Alex, the answer is "What is another
name for Ursine Child?"[/cright][Q] I enchanted my opponent's [c]Bear Cub[/c] with my [c]Sigil of Sleep[/c]. If the Bear Cub goes unblocked attacking me, who chooses which creature I control gets returned to its owner's hand?[/Q]

[A] You get to choose as you control the Sigil of Sleep.[/A]

[hr]
[Q] I attack with [c]Geist of Saint Traft[/c] while I control [c]Anointed Procession[/c]. When combat ends, will both Angel tokens get exiled or do I get to keep one?[/Q]

[A] Both still get exiled. Saint Traft's triggered ability still made both tokens and tracks them to be exiled at the end of combat. Procession doesn't make tokens, it just changes how many tokens other cards and abilities make.[/A]

[hr]
[Q] I attack with [c]Geist of Saint Traft[/c], but before blocks, I cast [c]Eyes in the Skies[/c] choosing to populate the Angel token. When combat ends, will both Angel tokens get exiled or do I get to keep one?[/Q]

[A] You get to keep the one made by Eyes in the Skies. Unlike Anointed Procession from the last question, Eyes in the Skies makes the token and the token has no link to the ability that made the original Angel token. This includes that the second Angel token is not attacking, is not tapped, and it won't get exiled at the end of combat.[/A]

[hr]
[Q] I don't understand what [c]Emrakul, the World Anew[/c] has protection from. Can you clarify this?[/Q]

[A] Emrakul has protection from spells and separately protection from permanents cast "this turn". This turn is whichever turn it currently is, even if it's been many turns since Emrakul herself was cast. 

So for example, if I cast [c]Visara the Dreadful[/c] and gave her some [c]Lightning Greaves[/c] the same turn I cast her, she can't target and destroy Emrakul that turn. However, if I pass to my opponent's turn, Visara can now kill Emrakul as it's no longer the same turn Visara was cast. If I used [c]Goryo's Vengeance[/c] to reanimate Visara, then Visara could kill Emrakul that same turn as Visara wasn't cast at all.

Emrakul's protection ability acts similar to having protection from Auras. Auras when cast target what they will enchant, so Emrakul is safe from most Auras. However, on later turns, an Aura could be moved from its original target, to enchant Emrakul. For example, you could move [c]Pacifism[/c] from a 1/1 Goblin token to Emrakul using [c]Crown of the Ages[/c]. 
[/A]

[hr]
[Q] I cast [c]Season of Weaving[/c] choosing the two paws mode (to copy my [c]Arcanis the Omnipotent[/c]) and the three paws mode. Will I have to put one Arcanis into my graveyard due to the legend rule before bouncing all nonland and nontoken permanents?[/Q]

[A] You will not and you can not. The legend rule is a state based action. It's never checked in the middle of resolving a spell. You will put the original Arcanis into your hand before Season of Weaving fully resolves. The legend rule doesn't come into play during this situation.[/A]

[hr]
[Q] If I cast [c]Righteous Confluence[/c] choosing mode two once to exile my opponent's [c]Leyline of Sanctity[/c] and mode three twice to gain 10 total life, and the opponent responds by [c]Boomerang[/c]ing the Leyline back to their hand, do I still gain 10 life?[/Q]

[A] You do not. Righteous Confluence had only one target, Leyline of Sanctity, and that target has disappeared. If a spell loses all of its targets by the time it goes to resolve, it fizzles and none of its effects happen. [/A]

[hr]
[Q] I just cast [c]Cultivate[/c] putting a [c]Swamp[/c] onto the battlefield and a [c]Forest[/c] into my hand. I haven't played a land from my hand yet this turn. Can I still play the Forest as my land drop for the turn, or did the Swamp use up my land drop for the turn?[/Q]

[A] You may still play the Forest. Putting a land onto the battlefield with a spell or ability doesn't use up your one land drop per turn.[/A]

[hr]

Each Question in my last article had a hidden reference to a different pop-culture pig. With ten different questions there were ten different pigs. The spoiler below has the answer key to those references.
[spoiler]
Question 1) Charlotte's Web - Wilbur the Pig and Charlotte the Spider
Question 2) Jim Henson's Muppets - Miss Piggy, Kermit the Frog, and Fozzie Bear.
Question 3) The Three Little Pigs and the Big Bad Wolf
Question 4) Angry Birds Video Game (Birds vs Pigs)
Question 5) Looney Tunes - Bugs Bunny, Porky Pig, and Daffy Duck
Question 6) Babe the 1995 Movie - Babe the Pig, Sheep for Babe to herd, and Farmer Arthur Hoggett
Question 7) Legend of Zelda - Link an Elf Warrior, Zelda an Elf Noble, and Ganon (pig form of Ganondorf)
Question 8) The Lion King - Timon the Meerkat, Pumbaa the Warthog, and some Hyenas for them to fight.
Question 9) Toy Story - Woody the Cowboy, Buzz the Spaceman, Rex the T-Rex, and Hamm the Piggy Bank.
Question 10) Winnie the Pooh - Pooh the Bear, Tigger the Tiger, and Piglet the Pig.[/spoiler]

- Justin Hovdenes AKA Hovey
Level 2 Magic Judge
Rapid City, SD

{'replies': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}], 'writeControl': {'requiredRevisionId': 'ALBJ4LsPa7aFSC00gFRAKkLEQnBrbtoKHU2y4dT7qxRhOpUd00nYiqWrR9VlSiWS1tYUDKeFIkSqznCMTzRSGw'}, 'documentId': '1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo'}
The Google doc at this URL has been modified with the desired content of length 15

> ||@everyone|| [Notre Google doc partagé](https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing) a été mis à jour, avec le brouillon de traduction pour [le nouvel article hebdomadaire](https://www.cranial-insertion.com/article/4376).

Cet article s'intitule "Cranial Insertion: All in Jeopardy (Can you answer in the form of a question?)", a été publié le Mon, 31 Mar 2025 00:00:00 EDT, et son ID est 4376.

> Envoyé automatiquement depuis un notebook Python executé sur Google Colab! Code by @Naereen


Starting to send the following message to my Discord server:
||@everyone|| [Notre Google doc partagé](https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing) a été mis à jour, avec le brouillon de traduction pour [le nouvel article hebdomadaire](https://www.cranial-insertion.com/article/4376).

Cet article s'intitule "Cranial Insertion: All in Jeopardy (Can you answer in the form of a question?)", a été publié le Mon, 31 Mar 2025 00:00:00 EDT, et son ID est 4376.

> Envoyé automatiquement depuis un notebook Python executé sur Google Colab! Code by @Naereen

Success!
CPU times: user 520 ms, sys: 26.2 ms, total: 546 ms
Wall time: 2.39 s


''

------------

## Old notebook, on finetuning Google's T5 model: TODO adapt this to finetune [Helsinki-NLP/opus-mt-tc-big-en-fr](https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-fr) for Magic: the Gathering keywords and cardnames

See:

- <https://huggingface.co/docs/transformers/tasks/translation>
- <https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation-tf.ipynb>

In [2]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

## Translation

In [3]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/1JvfrvZgi6c?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.11/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Translation converts a sequence of text from one language to another. It is one of several tasks you can formulate as a sequence-to-sequence problem, a powerful framework for returning some output from an input, like translation or summarization. Translation systems are commonly used for translation between different language texts, but it can also be used for speech or some combination in between like text-to-speech or speech-to-text.

This guide will show you how to:

1. Finetune [T5](https://huggingface.co/t5-small) on the English-French subset of the [OPUS Books](https://huggingface.co/datasets/opus_books) dataset to translate English text to French.
2. Use your finetuned model for inference.

<Tip>
The task illustrated in this tutorial is supported by the following model architectures:

<!--This tip is automatically generated by `make fix-copies`, do not fill manually!-->

[BART](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bart), [BigBird-Pegasus](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bigbird_pegasus), [Blenderbot](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/blenderbot), [BlenderbotSmall](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/blenderbot-small), [Encoder decoder](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/encoder-decoder), [FairSeq Machine-Translation](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/fsmt), [GPTSAN-japanese](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gptsan-japanese), [LED](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/led), [LongT5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/longt5), [M2M100](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/m2m_100), [Marian](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/marian), [mBART](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mbart), [MT5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mt5), [MVP](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mvp), [NLLB](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nllb), [NLLB-MOE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nllb-moe), [Pegasus](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/pegasus), [PEGASUS-X](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/pegasus_x), [PLBart](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/plbart), [ProphetNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/prophetnet), [SwitchTransformers](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/switch_transformers), [T5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/t5), [XLM-ProphetNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm-prophetnet)

<!--End of the generated tip-->

</Tip>

Before you begin, make sure you have all the necessary libraries installed:

```bash
pip install transformers datasets evaluate sacrebleu
```

We encourage you to login to your Hugging Face account so you can upload and share your model with the community. When prompted, enter your token to login:

In [4]:
!pip install transformers datasets evaluate sacrebleu

In [5]:
from huggingface_hub import notebook_login

notebook_login()

## Load OPUS Books dataset

Start by loading the English-French subset of the [OPUS Books](https://huggingface.co/datasets/opus_books) dataset from the 🤗 Datasets library:

In [6]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-fr")

Split the dataset into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [7]:
books = books["train"].train_test_split(test_size=0.2)

Then take a look at an example:

In [8]:
books["train"][0]

{'id': '57272',
 'translation': {'en': '"Do you know that you made a great figure on Christmas Day at the bead of your company of the chamber of accounts, Monsieur President?"',
  'fr': '– Savez-vous que vous aviez bien grande mine, le jour de Noël, à la tête de votre compagnie de la chambre des Comptes, monsieur le président ?'}}

`translation`: an English and French translation of the text.

## Preprocess

In [9]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/XAR8jnZZuUs?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.11/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


The next step is to load a T5 tokenizer to process the English-French language pairs:

In [10]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The preprocessing function you want to create needs to:

1. Prefix the input with a prompt so T5 knows this is a translation task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2. Tokenize the input (English) and target (French) separately because you can't tokenize French text with a tokenizer pretrained on an English vocabulary.
3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [26]:
source_lang = "en"
target_lang = "fr"
prefix = "Translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True, padding=True) # Add padding=True to the tokenizer call
    # Add decoder_input_ids to the model inputs, ensuring it's a simple list
    # Instead of copying input_ids directly, ensure decoder_input_ids
    # are appropriately formatted for sequence-to-sequence tasks.
    # For example, you may need to shift the input_ids and add special tokens
    # (like a start-of-sequence token) for the decoder.
    # Refer to the documentation or tutorials for T5 or the DataCollatorForSeq2Seq
    # for how to correctly format decoder_input_ids
    #
    # Assuming decoder_input_ids should be a shifted version of input_ids:
    decoder_input_ids = [ [tokenizer.bos_token_id] + ids[:-1] for ids in model_inputs["input_ids"] ]
    model_inputs["decoder_input_ids"] = decoder_input_ids
    return model_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) method. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [27]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [28]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [SacreBLEU](https://huggingface.co/spaces/evaluate-metric/sacrebleu) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [29]:
import evaluate

metric = evaluate.load("sacrebleu")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the SacreBLEU score:

In [30]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

<Tip>

If you aren't familiar with finetuning a model with Keras, take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-a-tensorflow-model-with-keras)!

</Tip>

To finetune a model in TensorFlow, start by setting up an optimizer function, learning rate schedule, and some training hyperparameters:

In [31]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

Then you can load T5 with [TFAutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.TFAutoModelForSeq2SeqLM):

In [32]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Convert your datasets to the `tf.data.Dataset` format with [prepare_tf_dataset()](https://huggingface.co/docs/transformers/main/en/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset):

In [33]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_books["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_books["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`decoder_input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

Trying to fix the issue I got on the first experiment... Thanks to Google's Gemini for the suggestions.

In [ ]:
tf_validation_set = model.prepare_tf_dataset(
    tokenized_books["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
# Ensure the validation set is prefetched for faster evaluation
tf_validation_set = tf_validation_set.prefetch(tf.data.AUTOTUNE)

Configure the model for training with [`compile`](https://keras.io/api/models/model_training_apis/#compile-method). Note that Transformers models all have a default task-relevant loss function, so you don't need to specify one unless you want to:

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

The last two things to setup before you start training is to compute the SacreBLEU metric from the predictions, and provide a way to push your model to the Hub. Both are done by using [Keras callbacks](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/keras_callbacks).

Pass your `compute_metrics` function to [KerasMetricCallback](https://huggingface.co/docs/transformers/main/en/main_classes/keras_callbacks#transformers.KerasMetricCallback):

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

Specify where to push your model and tokenizer in the [PushToHubCallback](https://huggingface.co/docs/transformers/main/en/main_classes/keras_callbacks#transformers.PushToHubCallback):

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    # output_dir="my_awesome_opus_books_model",
    output_dir="my_awesome_opus_books_model",
    tokenizer=tokenizer,
)

Then bundle your callbacks together:

In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

Finally, you're ready to start training your model! Call [`fit`](https://keras.io/api/models/model_training_apis/#fit-method) with your training and validation datasets, the number of epochs, and your callbacks to finetune the model:

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1, callbacks=callbacks)

Once training is completed, your model is automatically uploaded to the Hub so everyone can use it!

<Tip>

For a more in-depth example of how to finetune a model for translation, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation.ipynb)
or [TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation-tf.ipynb).

</Tip>

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Come up with some text you'd like to translate to another language. For T5, you need to prefix your input depending on the task you're working on. For translation from English to French, you should prefix your input as shown below:

In [ ]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

text2 = "translate English to French but ignore short HTML tags: [a]I'm afraid not. Bloodletter of Aclazotz causes your opponent to lose twice the amount of life due the damage that Kotis deals to them, but the X in Kotis's ability is the amount of damage it dealt, not the amount of life that was lost as a result of the damage.[/a]"

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for translation with your model, and pass your text to it:

In [ ]:
from transformers import pipeline

# translator = pipeline("translation", model="my_awesome_opus_books_model")
translator = pipeline("translation", model="google-t5/t5-small")

In [ ]:
translator(text)

In [ ]:
translator(text2)

You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return the `input_ids` as TensorFlow tensors:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_awesome_opus_books_model")
inputs = tokenizer(text, return_tensors="tf").input_ids

Use the [generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.TFGenerationMixin.generate) method to create the translation. For more details about the different text generation strategies and parameters for controlling generation, check out the [Text Generation](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/text_generation) API.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("my_awesome_opus_books_model")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

Decode the generated token ids back into text:

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)